In [1]:
from __future__ import division
import pandas as pd,re,numpy as np
from functools import reduce
from pyomo.environ import *
import pyomo.environ as pyo
import itertools as it

In [2]:
solution = pd.read_csv("Weekly_optimiser_2_results.csv",index_col=0)
solution["EDLP_Perc"] =solution["EDLP_Price"]/solution["BasePrice"] 
solution.sort_values(by=["Product","week"],inplace=True)
solution.head(7)

,Product,week,EDLP_dollar_sales,TPR_dollar_sales,Event_flag,Units_sold,EDLP_Price,TPR_discount,BasePrice,TPR_Spend,EDLP_Spend,EDLP_Perc
0,Product1_Retailer,0,11844.57809,0.0,1,5988.740788,1.98,0.0,1.99,0.0,73.016075,0.994975
0,Product1_Retailer,1,11963.61808,0.0,1,6048.928629,1.98,0.0,1.99,0.0,73.749896,0.994975
0,Product1_Retailer,2,12081.52194,0.0,1,6108.604704,1.98,0.0,1.99,0.0,74.601424,0.994975
0,Product1_Retailer,3,11963.61808,0.0,1,6048.928629,1.98,0.0,1.99,0.0,73.749896,0.994975
0,Product1_Retailer,4,12203.71982,0.0,1,6170.368979,1.98,0.0,1.99,0.0,75.314447,0.994975
0,Product1_Retailer,5,12082.29078,0.0,1,6108.972786,1.98,0.0,1.99,0.0,74.565059,0.994975
0,Product1_Retailer,6,12203.71982,0.0,1,6170.368979,1.98,0.0,1.99,0.0,75.314447,0.994975


In [3]:
Tot_Week = 52
Tot_Prod = 6

In [4]:
edlp = list(solution["EDLP_Perc"])

EDLP_Sol = [edlp[j:j+Tot_Week] for j in range(0,len(solution.index),Tot_Week) ]

In [5]:
tpr = list(solution["TPR_discount"])

TPR_Sol = [tpr[j:j+Tot_Week] for j in range(0,len(solution.index),Tot_Week) ]

In [6]:
flag = list(solution["Event_flag"])

FLAG_Sol = [flag[j:j+Tot_Week] for j in range(0,len(solution.index),Tot_Week) ]

In [7]:
coef_df = pd.read_csv("coefficient_matrix.csv")

annual_df = pd.read_csv("AnnualOptimizerInputData.csv")

quarter_df = pd.read_csv("LastQuarterData.csv")

In [8]:
Tot_Prod = coef_df["Modelled_Product"].nunique()

Tot_Week = coef_df["wk"].nunique()

In [9]:
# Base_Price_Products

Base_Price = [i for i in solution["BasePrice"] ]

Base_Price = [Base_Price[j:j+Tot_Week] for j in range(0,len(solution.index),Tot_Week) ]

# print(Base_Price)

In [10]:
con = [i for i in coef_df["Constant"]]

Intercepts = [con[j:j+Tot_Prod] for j in range(0,len(con),Tot_Prod)]

# print(Intercepts)

In [11]:
EDLP_Coef = np.array(coef_df[[i for i in coef_df.columns if i.count("EDLP")==1 ]])
print(EDLP_Coef[0:10])

[[-2.70352116  0.          0.          0.          0.          0.67533423]
 [ 0.         -3.71640368  0.          0.          0.          0.        ]
 [ 0.          0.         -0.14812202  0.          0.          0.        ]
 [ 0.          0.          0.         -0.96070924  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -5.73144924]
 [-2.70352116  0.          0.          0.          0.          0.67533423]
 [ 0.         -3.71640368  0.          0.          0.          0.        ]
 [ 0.          0.         -0.14812202  0.          0.          0.        ]
 [ 0.          0.          0.         -0.96070924  0.          0.        ]]


In [12]:
TPR_Coef = np.array(coef_df[[i for i in coef_df.columns if i.count("TPR")==1 ]])
# print(TPR_Coef)

In [13]:
# Available EDLP Interactions pairs

EDLP_int = [re.findall("[0-9]",i) for i in coef_df.columns if i.count("EDLP")>1 ] 

EDLP_Interactions = []

for i in EDLP_int:

    temp=[]
    
    for j in i:
        
        temp.append(int(j)-1)
        
    EDLP_Interactions.append(temp)

EDLP_Interactions

[[0, 5]]

In [14]:
# Available TPR Interactions pairs

TPR_int = [re.findall("[0-9]",i) for i in coef_df.columns if i.count("TPR")>1 ] 

TPR_Interactions = []

for i in TPR_int:

        temp=[]
        
        for j in i:
        
            temp.append(int(j)-1)
        
        TPR_Interactions.append(temp)

# TPR_Interactions

In [15]:
# EDLP_Interaction_Coef_Values

EDLP_Int_Coef_Values = {}

for col in coef_df.columns:
    
    if col.count("EDLP")>1 :
        
        Pair_name ="_".join([str(int(i)-1) for i in re.findall("[0-9]",col)])
                
        EDLP_Int_Coef_Values[Pair_name] = list(coef_df[col])

In [16]:
# TPR_Interaction_Coef_Values

TPR_Int_Coef_Values = {}

for col in coef_df.columns:
    
    if col.count("TPR")>1 :
        
        Pair_name ="_".join([str(int(i)-1) for i in re.findall("[0-9]",col)])
                
        TPR_Int_Coef_Values[Pair_name] = list(coef_df[col])

In [206]:
def Retailer_Unit_Sales_Fn(EDLP,TPR,FLAG,P,W):
    
    Self = exp( (log(EDLP[P][W] * Base_Price[P][W]) * EDLP_Coef[P][P]) * FLAG[P][W]
                     
                     + (log(Base_Price[P][W]) * EDLP_Coef[P][P] + TPR[P][W] * TPR_Coef[P][P]) * (1 - FLAG[P][W])
                     
                     + Intercepts[W][P])
    
    Comp = Competitor_Unit_Effect_Fn(EDLP,TPR,FLAG,P,W)
    
    EDLP_Inter_Val = EDLP_Interaction_Unit_Effect_Fn(EDLP,TPR,FLAG,P,W)
    
    TPR_Inter_Val = TPR_Interaction_Unit_Effect_Fn(EDLP,TPR,FLAG,P,W)    
    
    Unit_Sales = Self * Comp * EDLP_Inter_Val * TPR_Inter_Val
    
#     print(f"\n\nSelf: {Self}\nComp: {Comp}\nEDLP_Inter: {EDLP_Inter_Val}\nTPR_Inter: {TPR_Inter_Val}")
    
    return Unit_Salesles

In [196]:
def Competitor_Unit_Effect_Fn(EDLP,TPR,FLAG,Cur_Ret,Wk):
    
    Comp_Retailers_Unit_Sales = [exp((log(EDLP[i][Wk] * Base_Price[i][Wk]) * EDLP_Coef[Cur_Ret][i]) * FLAG[i][Wk] +
                                     
                                     (log(Base_Price[i][Wk]) * EDLP_Coef[Cur_Ret][i] + TPR[i][Wk] * TPR_Coef[Cur_Ret][i]) *
                                     
                                     (1 - FLAG[i][Wk])) for i in range(Tot_Prod) if i != Cur_Ret]
    
    
    return reduce(lambda x,y:x*y, Comp_Retailers_Unit_Sales,1)

In [291]:
def EDLP_Interaction_Unit_Effect_Fn(EDLP,TPR,FLAG,P,Wk):
    
    EDLP_Interaction_Unit_Effect = [1]
    
    for Inter in EDLP_Interactions:
    
            if P in Inter:
                
                Pr_ind1, Pr_ind2 = Inter
                
                Comb = str(Pr_ind1) + "_" + str(Pr_ind2)
                
                Val = exp((Base_Price[Pr_ind1][Wk] * Base_Price[Pr_ind2][Wk] * EDLP_Int_Coef_Values[Comb][P] *
                       
                        (1 - FLAG[Pr_ind1][Wk]) * (1 - FLAG[Pr_ind2][Wk])) +
                       
                       (Base_Price[Pr_ind1][Wk] * log(EDLP[Pr_ind2][Wk] * Base_Price[Pr_ind2][Wk]) *
                        
                        EDLP_Int_Coef_Values[Comb][P] * (1 - FLAG[Pr_ind1][Wk]) * (FLAG[Pr_ind2][Wk]) ) +
                       
                       ( log(Base_Price[Pr_ind1][Wk] * EDLP[Pr_ind1][Wk]) * Base_Price[Pr_ind2][Wk] * 
                        
                        EDLP_Int_Coef_Values[Comb][P] * (FLAG[Pr_ind1][Wk]) * (1 - FLAG[Pr_ind2][Wk])) +
                       
                       (log(Base_Price[Pr_ind1][Wk] * EDLP[Pr_ind1][Wk]) * log(Base_Price[Pr_ind2][Wk] * 
                        
                        EDLP[Pr_ind2][Wk]) * EDLP_Int_Coef_Values[Comb][P] * 
                       
                        (FLAG[Pr_ind1][Wk]) * (FLAG[Pr_ind2][Wk])) )
                        
                EDLP_Interaction_Unit_Effect.append(Val)
               
    return reduce(lambda x,y: x*y, EDLP_Interaction_Unit_Effect)

In [292]:
def TPR_Interaction_Unit_Effect_Fn(EDLP,TPR,FLAG,P,Wk):
    
    TPR_Interaction_Unit_Effect = [1]
    
    for Inter in TPR_Interactions:
    
            if P in Inter:
                
                Pr_ind1, Pr_ind2 = Inter
                
                Comb = str(Pr_ind1) + "_" + str(Pr_ind2)
                
                Val = exp(( TPR[Pr_ind1][Wk] * TPR[Pr_ind2][Wk] *
                           
                           TPR_Int_Coef_Values[Comb][P] * (1 - FLAG[Pr_ind1][Wk]) * (1 - FLAG[Pr_ind2][Wk])) ) 
                          
                TPR_Interaction_Unit_Effect.append(Val)
    
    return reduce(lambda x,y: x*y, TPR_Interaction_Unit_Effect)

In [293]:
def Retailer_Price_Fn(EDLP,TPR,FLAG,P,W):
    
    Price = ((Base_Price[P][W] * (1 - TPR[P][W]) * (1 - FLAG[P][W])) + 
             
             (EDLP[P][W] * Base_Price[P][W] * FLAG[P][W]) )
    
    return Price

In [294]:
def Dollar_Sales_Fn(EDLP,TPR,FLAG):
    
#     print(( [ Retailer_Unit_Sales_Fn(Model,P,W) * Retailer_Price_Fn(Model,P,W) for W in Model.Wk_index
                 
#                  for P in Model.PPG_index] ))
    
    return ( [ Retailer_Unit_Sales_Fn(EDLP,TPR,FLAG,P,W) * Retailer_Price_Fn(EDLP,TPR,FLAG,P,W) for W in range(Tot_Week)
                 
                 for P in range(Tot_Prod)] )

In [295]:
def EDLP_Total_Trade_Spent_Fn(EDLP,TPR,FLAG):
    
    return [ (Base_Price[P][W] - Retailer_Price_Fn(EDLP,TPR,FLAG,P,W)) * Retailer_Unit_Sales_Fn(EDLP,TPR,FLAG,P,W) *  (FLAG_Sol[P][W])
            for W in range(Tot_Week) for P in range(Tot_Prod) ]

In [296]:
def TPR_Total_Trade_Spent_Fn(EDLP,TPR,FLAG):
    
    return [ (Base_Price[P][W] - Retailer_Price_Fn(EDLP,TPR,FLAG,P,W)) * Retailer_Unit_Sales_Fn(EDLP,TPR,FLAG,P,W) *  (1-FLAG_Sol[P][W])
            for W in range(Tot_Week) for P in range(Tot_Prod) ]

In [297]:
EDLP_Spend = EDLP_Total_Trade_Spent_Fn(EDLP_Sol,TPR_Sol,FLAG_Sol)

In [298]:
TPR_Spend = TPR_Total_Trade_Spent_Fn(EDLP_Sol,TPR_Sol,FLAG_Sol)

In [312]:
Units_Sold

[5977.307764244694,
 12071.770656280463,
 2836.0314836296807,
 11866.504340397743,
 8009.743315153029,
 43201.624157294274,
 6016.312859948593,
 12127.615359258763,
 2861.0450295223227,
 11888.724975418927,
 8009.743315153029,
 43985.09694623006,
 6114.143353412895,
 12399.185851227447,
 2901.802988651588,
 12092.25637375386,
 8009.743315153029,
 45751.42966662588,
 6005.072687416624,
 12125.275682867212,
 2852.7622545745453,
 11887.663003050598,
 8002.081825532082,
 43747.236311130386,
 6135.127129426307,
 12615.631537888099,
 2906.825938273531,
 12039.155424053333,
 8119.347097085027,
 45633.36332132078,
 6114.692489286044,
 12411.026524184625,
 2900.7004868932395,
 11993.166268108045,
 8052.0733801454035,
 45288.42589228608,
 6128.999604296218,
 12419.452374418264,
 2909.0651201952055,
 12005.193044870894,
 8045.930913239294,
 45573.27594003536,
 6108.736318485139,
 12350.880465630593,
 2899.488569308624,
 12015.273457727832,
 8063.153313879238,
 45194.266469708964,
 6209.2471700490

In [313]:
Dollar_Sales = []
Dollar_Sales.extend(Dollar_Sales_Fn(EDLP_Sol,TPR_Sol,FLAG_Sol))

In [314]:
Dollar_Sales

[11835.069373204495,
 36456.747381967,
 10011.191137212772,
 35718.1780645972,
 27233.127271520298,
 42769.60791572133,
 11912.299462698213,
 36625.39838496147,
 10099.488954213799,
 35785.062176010964,
 27233.127271520298,
 43545.245976767765,
 12106.003839757532,
 37445.54127070689,
 10243.364549940105,
 36276.76912126158,
 27233.127271520298,
 45293.91536995962,
 11890.043921084914,
 36618.33256225898,
 10070.250758648144,
 35781.8656391823,
 27207.078206809078,
 43309.763948019085,
 12147.551716264088,
 38099.20724442206,
 10261.095562105564,
 36237.857826400534,
 26331.04263584674,
 45177.02968810757,
 12107.091128786367,
 37481.30010303757,
 10239.472718733135,
 36099.430467005215,
 27377.04949249437,
 44835.541633363224,
 12135.419216506512,
 37506.74617074316,
 10268.999874289075,
 36135.63106506139,
 27356.1651050136,
 45117.54318063501,
 12095.297910600575,
 37299.65900620439,
 10235.194649659443,
 36045.820373183495,
 27414.72126718941,
 44742.323805011874,
 12294.3093966971

In [315]:
def Retailer_Unit_Sales_Fn1(EDLP,TPR,FLAG,P,W):
    
    Self = exp( (log(EDLP[P][W] * Base_Price[P][W]) * EDLP_Coef[P][P]) * FLAG[P][W]
                     
                     + (log(Base_Price[P][W]) * EDLP_Coef[P][P] + TPR[P][W] * TPR_Coef[P][P]) * (1 - FLAG[P][W])
                     
                     + Intercepts[W][P])
    
    Comp = Competitor_Unit_Effect_Fn(EDLP,TPR,FLAG,P,W)
    
    EDLP_Inter_Val = EDLP_Interaction_Unit_Effect_Fn(EDLP,TPR,FLAG,P,W)
    
    TPR_Inter_Val = TPR_Interaction_Unit_Effect_Fn(EDLP,TPR,FLAG,P,W)    
    
    Unit_Sales = Self * Comp * EDLP_Inter_Val * TPR_Inter_Val
    
#     print(f"\n\nSelf: {Self}\nComp: {Comp}\nEDLP_Inter: {EDLP_Inter_Val}\nTPR_Inter: {TPR_Inter_Val}")
    
    return Unit_Sales,Comp,EDLP_Inter_Val,TPR_Inter_Val,Self

In [316]:
Competetor_Effect = []

EDLP_Multiplicative_Effect = []

TPR_Multiplicative_Effect = []

Self_Effect = []


for W in range(1,Tot_Week+1):
    
    for P in range(1,Tot_Prod+1):
        
        res = (Retailer_Unit_Sales_Fn1(EDLP_Sol,TPR_Sol,FLAG_Sol,P-1,W-1))
        
        Competetor_Effect.append(res[1])
        EDLP_Multiplicative_Effect.append(res[2])
        TPR_Multiplicative_Effect.append(res[3])        
        Self_Effect.append(res[4])        

        

In [336]:
Solution_Wo_Optimizer = pd.DataFrame({"Dollar_Sales":Dollar_Sales,"Units_sold":Units_Sold,"EDLP_Spend":EDLP_Spend,
                                     "TPR_Spend":TPR_Spend},index=[f"P{i+1}_W{j+1}" for j in range(Tot_Week) for i in range(Tot_Prod)])
Solution_Wo_Optimizer["Flag"] = list(solution["Event_flag"])

Solution_Wo_Optimizer["EDLP_Dollar_Sales"] = Solution_Wo_Optimizer["Dollar_Sales"] * Solution_Wo_Optimizer["Flag"]

Solution_Wo_Optimizer["TPR_Dollar_Sales"] = Solution_Wo_Optimizer["Dollar_Sales"] * (1-Solution_Wo_Optimizer["Flag"])

Solution_Wo_Optimizer["Competetor_Effect"] = Competetor_Effect

Solution_Wo_Optimizer["EDLP_Multiplicative_Effect"] = EDLP_Multiplicative_Effect

Solution_Wo_Optimizer["TPR_Multiplicative_Effect"] = TPR_Multiplicative_Effect

Solution_Wo_Optimizer["Self_Effect"] = Self_Effect

Solution_Wo_Optimizer

,Dollar_Sales,Units_sold,EDLP_Spend,TPR_Spend,Flag,EDLP_Dollar_Sales,TPR_Dollar_Sales,Competetor_Effect,EDLP_Multiplicative_Effect,TPR_Multiplicative_Effect,Self_Effect
P1_W1,11835.069373,5977.307764,59.773078,0.0,1,11835.069373,0.0,0.993236,0.998767,1,6025.447889
P2_W1,36456.747382,12071.770656,362.153120,0.0,1,36456.747382,0.0,1.000000,1.000000,1,12071.770656
P3_W1,10011.191137,2836.031484,0.000000,0.0,1,10011.191137,0.0,1.000000,1.000000,1,2836.031484
P4_W1,35718.178065,11866.504340,0.000000,0.0,1,35718.178065,0.0,1.000000,1.000000,1,11866.504340
P5_W1,27233.127272,8009.743315,400.487166,0.0,1,27233.127272,0.0,1.000000,1.000000,1,8009.743315
...,...,...,...,...,...,...,...,...,...,...,...
P2_W52,37778.646088,12509.485459,375.284564,0.0,1,37778.646088,0.0,1.000000,1.000000,1,12509.485459
P3_W52,10185.440422,2885.393887,0.000000,0.0,1,10185.440422,0.0,1.000000,1.000000,1,2885.393887
P4_W52,35905.578331,11928.763565,0.000000,0.0,1,35905.578331,0.0,1.000000,1.000000,1,11928.763565
P5_W52,26223.861371,7712.900403,385.645020,0.0,1,26223.861371,0.0,1.000000,1.000000,1,7712.900403


In [318]:
Solution_Wo_Optimizer["Self_Vs_Competetor"] = Solution_Wo_Optimizer["Self_Effect"] * Solution_Wo_Optimizer["Competetor_Effect"]

Solution_Wo_Optimizer["Self_Vs_ED_Multiplicative"] = Solution_Wo_Optimizer["Self_Effect"] * Solution_Wo_Optimizer["EDLP_Multiplicative_Effect"]


In [319]:
Solution_Wo_Optimizer[["Units_sold","Competetor_Effect","EDLP_Multiplicative_Effect","TPR_Multiplicative_Effect","Self_Effect",
                     "Self_Vs_Competetor","Self_Vs_ED_Multiplicative"]]

,Units_sold,Competetor_Effect,EDLP_Multiplicative_Effect,TPR_Multiplicative_Effect,Self_Effect,Self_Vs_Competetor,Self_Vs_ED_Multiplicative
P1_W1,5977.307764,0.993236,0.998767,1,6025.447889,5984.689627,6018.015752
P2_W1,12071.770656,1.000000,1.000000,1,12071.770656,12071.770656,12071.770656
P3_W1,2836.031484,1.000000,1.000000,1,2836.031484,2836.031484,2836.031484
P4_W1,11866.504340,1.000000,1.000000,1,11866.504340,11866.504340,11866.504340
P5_W1,8009.743315,1.000000,1.000000,1,8009.743315,8009.743315,8009.743315
...,...,...,...,...,...,...,...
P2_W52,12509.485459,1.000000,1.000000,1,12509.485459,12509.485459,12509.485459
P3_W52,2885.393887,1.000000,1.000000,1,2885.393887,2885.393887,2885.393887
P4_W52,11928.763565,1.000000,1.000000,1,11928.763565,11928.763565,11928.763565
P5_W52,7712.900403,1.000000,1.000000,1,7712.900403,7712.900403,7712.900403


In [320]:
solution.sort_values(by=["week","Product"])

,Product,week,EDLP_dollar_sales,TPR_dollar_sales,Event_flag,Units_sold,EDLP_Price,TPR_discount,BasePrice,TPR_Spend,EDLP_Spend,EDLP_Perc
0,Product1_Retailer,0,11844.57809,0.0,1,5988.740788,1.98,0.0,1.99,0.0,73.016075,0.994975
1,Product2_Retailer,0,36458.98133,0.0,1,12073.583140,3.02,0.0,3.05,0.0,365.447258,0.990164
2,Product3_Retailer,0,10014.19163,0.0,1,2838.032025,3.53,0.0,3.53,0.0,4.061415,1.000000
3,Product4_Retailer,0,35741.85178,0.0,1,11874.370320,3.01,0.0,3.01,0.0,0.002881,1.000000
4,Product5_Retailer,0,27257.04577,0.0,1,8022.456895,3.40,0.0,3.45,0.0,420.430524,0.985507
...,...,...,...,...,...,...,...,...,...,...,...,...
1,Product2_Retailer,51,37569.32261,0.0,1,12441.278490,3.02,0.0,3.05,0.0,376.576782,0.990164
2,Product3_Retailer,51,10114.90800,0.0,1,2866.551117,3.53,0.0,3.53,0.0,4.017445,1.000000
3,Product4_Retailer,51,36101.06336,0.0,1,11993.709710,3.01,0.0,3.01,0.0,0.002877,1.000000
4,Product5_Retailer,51,26179.63417,0.0,1,7707.891861,3.40,0.0,3.45,0.0,412.592754,0.985507


In [321]:
5988.740788/5984.689627

1.0006769208183701

In [289]:
exp(log(Base_Price[0][0] * EDLP_Sol[0][0]) * log(Base_Price[5][0] * 
                        
                        EDLP_Sol[5][0]) * EDLP_Int_Coef_Values["0_5"][0] * 
                       
                        (FLAG_Sol[0][0]) * (FLAG_Sol[0][0]))

0.9987665421056567

In [351]:
(1.99 - (1.99*(1 - TPR_Sol[0][0])*(1 - FLAG_Sol[0][0]) + 1.99*EDLP_Sol[0][0]*FLAG_Sol[0][0]))*(exp(log(1.99*EDLP_Sol[0][0])*-2.703521163*FLAG_Sol[0][0] - 1.8603865588172197*(1 - FLAG_Sol[0][0]) + 10.55051387)*exp(log(0.99*EDLP_Sol[5][0])*0.675334228*FLAG_Sol[5][0] - 0.006787335804765123*(1 - FLAG_Sol[5][0]))*exp(0.3541748555565*(1 - FLAG_Sol[0][0])*(1 - FLAG_Sol[5][0]) + 1.99*log(0.99*EDLP_Sol[5][0])*0.179775065*(1 - FLAG_Sol[0][0])*FLAG_Sol[5][0] + log(1.99*EDLP_Sol[0][0])*0.99*0.179775065*FLAG_Sol[0][0]*(1 - FLAG_Sol[5][0]) + log(1.99*EDLP_Sol[0][0])*log(0.99*EDLP_Sol[5][0])*0.179775065*FLAG_Sol[0][0]*FLAG_Sol[5][0]))*FLAG_Sol[0][0] + (1.99 - (1.99*(1 - TPR_Sol[0][1])*(1 - FLAG_Sol[0][1]) + 1.99*EDLP_Sol[0][1]*FLAG_Sol[0][1]))*(exp(log(1.99*EDLP_Sol[0][1])*-2.703521163*FLAG_Sol[0][1] - 1.8603865588172197*(1 - FLAG_Sol[0][1]) + 10.557018199999998)*exp(log(0.99*EDLP_Sol[5][1])*0.675334228*FLAG_Sol[5][1] - 0.006787335804765123*(1 - FLAG_Sol[5][1]))*exp(0.3541748555565*(1 - FLAG_Sol[0][1])*(1 - FLAG_Sol[5][1]) + 1.99*log(0.99*EDLP_Sol[5][1])*0.179775065*(1 - FLAG_Sol[0][1])*FLAG_Sol[5][1] + log(1.99*EDLP_Sol[0][1])*0.99*0.179775065*FLAG_Sol[0][1]*(1 - FLAG_Sol[5][1]) + log(1.99*EDLP_Sol[0][1])*log(0.99*EDLP_Sol[5][1])*0.179775065*FLAG_Sol[0][1]*FLAG_Sol[5][1]))*FLAG_Sol[0][1] + (1.99 - (1.99*(1 - TPR_Sol[0][2])*(1 - FLAG_Sol[0][2]) + 1.99*EDLP_Sol[0][2]*FLAG_Sol[0][2]))*(exp(log(1.99*EDLP_Sol[0][2])*-2.703521163*FLAG_Sol[0][2] - 1.8603865588172197*(1 - FLAG_Sol[0][2]) + 10.57314828)*exp(log(0.99*EDLP_Sol[5][2])*0.675334228*FLAG_Sol[5][2] - 0.006787335804765123*(1 - FLAG_Sol[5][2]))*exp(0.3541748555565*(1 - FLAG_Sol[0][2])*(1 - FLAG_Sol[5][2]) + 1.99*log(0.99*EDLP_Sol[5][2])*0.179775065*(1 - FLAG_Sol[0][2])*FLAG_Sol[5][2] + log(1.99*EDLP_Sol[0][2])*0.99*0.179775065*FLAG_Sol[0][2]*(1 - FLAG_Sol[5][2]) + log(1.99*EDLP_Sol[0][2])*log(0.99*EDLP_Sol[5][2])*0.179775065*FLAG_Sol[0][2]*FLAG_Sol[5][2]))*FLAG_Sol[0][2] + (1.99 - (1.99*(1 - TPR_Sol[0][3])*(1 - FLAG_Sol[0][3]) + 1.99*EDLP_Sol[0][3]*FLAG_Sol[0][3]))*(exp(log(1.99*EDLP_Sol[0][3])*-2.703521163*FLAG_Sol[0][3] - 1.8603865588172197*(1 - FLAG_Sol[0][3]) + 10.55514817)*exp(log(0.99*EDLP_Sol[5][3])*0.675334228*FLAG_Sol[5][3] - 0.006787335804765123*(1 - FLAG_Sol[5][3]))*exp(0.3541748555565*(1 - FLAG_Sol[0][3])*(1 - FLAG_Sol[5][3]) + 1.99*log(0.99*EDLP_Sol[5][3])*0.179775065*(1 - FLAG_Sol[0][3])*FLAG_Sol[5][3] + log(1.99*EDLP_Sol[0][3])*0.99*0.179775065*FLAG_Sol[0][3]*(1 - FLAG_Sol[5][3]) + log(1.99*EDLP_Sol[0][3])*log(0.99*EDLP_Sol[5][3])*0.179775065*FLAG_Sol[0][3]*FLAG_Sol[5][3]))*FLAG_Sol[0][3] + (1.99 - (1.99*(1 - TPR_Sol[0][4])*(1 - FLAG_Sol[0][4]) + 1.99*EDLP_Sol[0][4]*FLAG_Sol[0][4]))*(exp(log(1.99*EDLP_Sol[0][4])*-2.703521163*FLAG_Sol[0][4] - 1.8603865588172197*(1 - FLAG_Sol[0][4]) + 10.57657441)*exp(log(0.99*EDLP_Sol[5][4])*0.675334228*FLAG_Sol[5][4] - 0.006787335804765123*(1 - FLAG_Sol[5][4]))*exp(0.3541748555565*(1 - FLAG_Sol[0][4])*(1 - FLAG_Sol[5][4]) + 1.99*log(0.99*EDLP_Sol[5][4])*0.179775065*(1 - FLAG_Sol[0][4])*FLAG_Sol[5][4] + log(1.99*EDLP_Sol[0][4])*0.99*0.179775065*FLAG_Sol[0][4]*(1 - FLAG_Sol[5][4]) + log(1.99*EDLP_Sol[0][4])*log(0.99*EDLP_Sol[5][4])*0.179775065*FLAG_Sol[0][4]*FLAG_Sol[5][4]))*FLAG_Sol[0][4] + (1.99 - (1.99*(1 - TPR_Sol[0][5])*(1 - FLAG_Sol[0][5]) + 1.99*EDLP_Sol[0][5]*FLAG_Sol[0][5]))*(exp(log(1.99*EDLP_Sol[0][5])*-2.703521163*FLAG_Sol[0][5] - 1.8603865588172197*(1 - FLAG_Sol[0][5]) + 10.57323809)*exp(log(0.99*EDLP_Sol[5][5])*0.675334228*FLAG_Sol[5][5] - 0.006787335804765123*(1 - FLAG_Sol[5][5]))*exp(0.3541748555565*(1 - FLAG_Sol[0][5])*(1 - FLAG_Sol[5][5]) + 1.99*log(0.99*EDLP_Sol[5][5])*0.179775065*(1 - FLAG_Sol[0][5])*FLAG_Sol[5][5] + log(1.99*EDLP_Sol[0][5])*0.99*0.179775065*FLAG_Sol[0][5]*(1 - FLAG_Sol[5][5]) + log(1.99*EDLP_Sol[0][5])*log(0.99*EDLP_Sol[5][5])*0.179775065*FLAG_Sol[0][5]*FLAG_Sol[5][5]))*FLAG_Sol[0][5] + (1.99 - (1.99*(1 - TPR_Sol[0][6])*(1 - FLAG_Sol[0][6]) + 1.99*EDLP_Sol[0][6]*FLAG_Sol[0][6]))*(exp(log(1.99*EDLP_Sol[0][6])*-2.703521163*FLAG_Sol[0][6] - 1.8603865588172197*(1 - FLAG_Sol[0][6]) + 10.57557515)*exp(log(0.99*EDLP_Sol[5][6])*0.675334228*FLAG_Sol[5][6] - 0.006787335804765123*(1 - FLAG_Sol[5][6]))*exp(0.3541748555565*(1 - FLAG_Sol[0][6])*(1 - FLAG_Sol[5][6]) + 1.99*log(0.99*EDLP_Sol[5][6])*0.179775065*(1 - FLAG_Sol[0][6])*FLAG_Sol[5][6] + log(1.99*EDLP_Sol[0][6])*0.99*0.179775065*FLAG_Sol[0][6]*(1 - FLAG_Sol[5][6]) + log(1.99*EDLP_Sol[0][6])*log(0.99*EDLP_Sol[5][6])*0.179775065*FLAG_Sol[0][6]*FLAG_Sol[5][6]))*FLAG_Sol[0][6] + (1.99 - (1.99*(1 - TPR_Sol[0][7])*(1 - FLAG_Sol[0][7]) + 1.99*EDLP_Sol[0][7]*FLAG_Sol[0][7]))*(exp(log(1.99*EDLP_Sol[0][7])*-2.703521163*FLAG_Sol[0][7] - 1.8603865588172197*(1 - FLAG_Sol[0][7]) + 10.57226354)*exp(log(0.99*EDLP_Sol[5][7])*0.675334228*FLAG_Sol[5][7] - 0.006787335804765123*(1 - FLAG_Sol[5][7]))*exp(0.3541748555565*(1 - FLAG_Sol[0][7])*(1 - FLAG_Sol[5][7]) + 1.99*log(0.99*EDLP_Sol[5][7])*0.179775065*(1 - FLAG_Sol[0][7])*FLAG_Sol[5][7] + log(1.99*EDLP_Sol[0][7])*0.99*0.179775065*FLAG_Sol[0][7]*(1 - FLAG_Sol[5][7]) + log(1.99*EDLP_Sol[0][7])*log(0.99*EDLP_Sol[5][7])*0.179775065*FLAG_Sol[0][7]*FLAG_Sol[5][7]))*FLAG_Sol[0][7] + (1.99 - (1.99*(1 - TPR_Sol[0][8])*(1 - FLAG_Sol[0][8]) + 1.99*EDLP_Sol[0][8]*FLAG_Sol[0][8]))*(exp(log(1.99*EDLP_Sol[0][8])*-2.703521163*FLAG_Sol[0][8] - 1.8603865588172197*(1 - FLAG_Sol[0][8]) + 10.58858327)*exp(log(0.99*EDLP_Sol[5][8])*0.675334228*FLAG_Sol[5][8] - 0.006787335804765123*(1 - FLAG_Sol[5][8]))*exp(0.3541748555565*(1 - FLAG_Sol[0][8])*(1 - FLAG_Sol[5][8]) + 1.99*log(0.99*EDLP_Sol[5][8])*0.179775065*(1 - FLAG_Sol[0][8])*FLAG_Sol[5][8] + log(1.99*EDLP_Sol[0][8])*0.99*0.179775065*FLAG_Sol[0][8]*(1 - FLAG_Sol[5][8]) + log(1.99*EDLP_Sol[0][8])*log(0.99*EDLP_Sol[5][8])*0.179775065*FLAG_Sol[0][8]*FLAG_Sol[5][8]))*FLAG_Sol[0][8] + (1.99 - (1.99*(1 - TPR_Sol[0][9])*(1 - FLAG_Sol[0][9]) + 1.99*EDLP_Sol[0][9]*FLAG_Sol[0][9]))*(exp(log(1.99*EDLP_Sol[0][9])*-2.703521163*FLAG_Sol[0][9] - 1.8603865588172197*(1 - FLAG_Sol[0][9]) + 10.58670597)*exp(log(0.99*EDLP_Sol[5][9])*0.675334228*FLAG_Sol[5][9] - 0.006787335804765123*(1 - FLAG_Sol[5][9]))*exp(0.3541748555565*(1 - FLAG_Sol[0][9])*(1 - FLAG_Sol[5][9]) + 1.99*log(0.99*EDLP_Sol[5][9])*0.179775065*(1 - FLAG_Sol[0][9])*FLAG_Sol[5][9] + log(1.99*EDLP_Sol[0][9])*0.99*0.179775065*FLAG_Sol[0][9]*(1 - FLAG_Sol[5][9]) + log(1.99*EDLP_Sol[0][9])*log(0.99*EDLP_Sol[5][9])*0.179775065*FLAG_Sol[0][9]*FLAG_Sol[5][9]))*FLAG_Sol[0][9] + (1.99 - (1.99*(1 - TPR_Sol[0][10])*(1 - FLAG_Sol[0][10]) + 1.99*EDLP_Sol[0][10]*FLAG_Sol[0][10]))*(exp(log(1.99*EDLP_Sol[0][10])*-2.703521163*FLAG_Sol[0][10] - 1.8603865588172197*(1 - FLAG_Sol[0][10]) + 10.57604441)*exp(log(0.99*EDLP_Sol[5][10])*0.675334228*FLAG_Sol[5][10] - 0.006787335804765123*(1 - FLAG_Sol[5][10]))*exp(0.3541748555565*(1 - FLAG_Sol[0][10])*(1 - FLAG_Sol[5][10]) + 1.99*log(0.99*EDLP_Sol[5][10])*0.179775065*(1 - FLAG_Sol[0][10])*FLAG_Sol[5][10] + log(1.99*EDLP_Sol[0][10])*0.99*0.179775065*FLAG_Sol[0][10]*(1 - FLAG_Sol[5][10]) + log(1.99*EDLP_Sol[0][10])*log(0.99*EDLP_Sol[5][10])*0.179775065*FLAG_Sol[0][10]*FLAG_Sol[5][10]))*FLAG_Sol[0][10] + (1.99 - (1.99*(1 - TPR_Sol[0][11])*(1 - FLAG_Sol[0][11]) + 1.99*EDLP_Sol[0][11]*FLAG_Sol[0][11]))*(exp(log(1.99*EDLP_Sol[0][11])*-2.703521163*FLAG_Sol[0][11] - 1.8603865588172197*(1 - FLAG_Sol[0][11]) + 10.57544812)*exp(log(0.99*EDLP_Sol[5][11])*0.675334228*FLAG_Sol[5][11] - 0.006787335804765123*(1 - FLAG_Sol[5][11]))*exp(0.3541748555565*(1 - FLAG_Sol[0][11])*(1 - FLAG_Sol[5][11]) + 1.99*log(0.99*EDLP_Sol[5][11])*0.179775065*(1 - FLAG_Sol[0][11])*FLAG_Sol[5][11] + log(1.99*EDLP_Sol[0][11])*0.99*0.179775065*FLAG_Sol[0][11]*(1 - FLAG_Sol[5][11]) + log(1.99*EDLP_Sol[0][11])*log(0.99*EDLP_Sol[5][11])*0.179775065*FLAG_Sol[0][11]*FLAG_Sol[5][11]))*FLAG_Sol[0][11] + (1.99 - (1.99*(1 - TPR_Sol[0][12])*(1 - FLAG_Sol[0][12]) + 1.99*EDLP_Sol[0][12]*FLAG_Sol[0][12]))*(exp(log(1.99*EDLP_Sol[0][12])*-2.703521163*FLAG_Sol[0][12] - 1.8603865588172197*(1 - FLAG_Sol[0][12]) + 10.57239458)*exp(log(0.99*EDLP_Sol[5][12])*0.675334228*FLAG_Sol[5][12] - 0.006787335804765123*(1 - FLAG_Sol[5][12]))*exp(0.3541748555565*(1 - FLAG_Sol[0][12])*(1 - FLAG_Sol[5][12]) + 1.99*log(0.99*EDLP_Sol[5][12])*0.179775065*(1 - FLAG_Sol[0][12])*FLAG_Sol[5][12] + log(1.99*EDLP_Sol[0][12])*0.99*0.179775065*FLAG_Sol[0][12]*(1 - FLAG_Sol[5][12]) + log(1.99*EDLP_Sol[0][12])*log(0.99*EDLP_Sol[5][12])*0.179775065*FLAG_Sol[0][12]*FLAG_Sol[5][12]))*FLAG_Sol[0][12] + (1.99 - (1.99*(1 - TPR_Sol[0][13])*(1 - FLAG_Sol[0][13]) + 1.99*EDLP_Sol[0][13]*FLAG_Sol[0][13]))*(exp(log(1.99*EDLP_Sol[0][13])*-2.703521163*FLAG_Sol[0][13] - 1.8603865588172197*(1 - FLAG_Sol[0][13]) + 10.58105812)*exp(log(0.99*EDLP_Sol[5][13])*0.675334228*FLAG_Sol[5][13] - 0.006787335804765123*(1 - FLAG_Sol[5][13]))*exp(0.3541748555565*(1 - FLAG_Sol[0][13])*(1 - FLAG_Sol[5][13]) + 1.99*log(0.99*EDLP_Sol[5][13])*0.179775065*(1 - FLAG_Sol[0][13])*FLAG_Sol[5][13] + log(1.99*EDLP_Sol[0][13])*0.99*0.179775065*FLAG_Sol[0][13]*(1 - FLAG_Sol[5][13]) + log(1.99*EDLP_Sol[0][13])*log(0.99*EDLP_Sol[5][13])*0.179775065*FLAG_Sol[0][13]*FLAG_Sol[5][13]))*FLAG_Sol[0][13] + (1.99 - (1.99*(1 - TPR_Sol[0][14])*(1 - FLAG_Sol[0][14]) + 1.99*EDLP_Sol[0][14]*FLAG_Sol[0][14]))*(exp(log(1.99*EDLP_Sol[0][14])*-2.703521163*FLAG_Sol[0][14] - 1.8603865588172197*(1 - FLAG_Sol[0][14]) + 10.56290342)*exp(log(0.99*EDLP_Sol[5][14])*0.675334228*FLAG_Sol[5][14] - 0.006787335804765123*(1 - FLAG_Sol[5][14]))*exp(0.3541748555565*(1 - FLAG_Sol[0][14])*(1 - FLAG_Sol[5][14]) + 1.99*log(0.99*EDLP_Sol[5][14])*0.179775065*(1 - FLAG_Sol[0][14])*FLAG_Sol[5][14] + log(1.99*EDLP_Sol[0][14])*0.99*0.179775065*FLAG_Sol[0][14]*(1 - FLAG_Sol[5][14]) + log(1.99*EDLP_Sol[0][14])*log(0.99*EDLP_Sol[5][14])*0.179775065*FLAG_Sol[0][14]*FLAG_Sol[5][14]))*FLAG_Sol[0][14] + (1.99 - (1.99*(1 - TPR_Sol[0][15])*(1 - FLAG_Sol[0][15]) + 1.99*EDLP_Sol[0][15]*FLAG_Sol[0][15]))*(exp(log(1.99*EDLP_Sol[0][15])*-2.703521163*FLAG_Sol[0][15] - 1.8603865588172197*(1 - FLAG_Sol[0][15]) + 10.57291005)*exp(log(0.99*EDLP_Sol[5][15])*0.675334228*FLAG_Sol[5][15] - 0.006787335804765123*(1 - FLAG_Sol[5][15]))*exp(0.3541748555565*(1 - FLAG_Sol[0][15])*(1 - FLAG_Sol[5][15]) + 1.99*log(0.99*EDLP_Sol[5][15])*0.179775065*(1 - FLAG_Sol[0][15])*FLAG_Sol[5][15] + log(1.99*EDLP_Sol[0][15])*0.99*0.179775065*FLAG_Sol[0][15]*(1 - FLAG_Sol[5][15]) + log(1.99*EDLP_Sol[0][15])*log(0.99*EDLP_Sol[5][15])*0.179775065*FLAG_Sol[0][15]*FLAG_Sol[5][15]))*FLAG_Sol[0][15] + (1.99 - (1.99*(1 - TPR_Sol[0][16])*(1 - FLAG_Sol[0][16]) + 1.99*EDLP_Sol[0][16]*FLAG_Sol[0][16]))*(exp(log(1.99*EDLP_Sol[0][16])*-2.703521163*FLAG_Sol[0][16] - 1.8603865588172197*(1 - FLAG_Sol[0][16]) + 10.54479686)*exp(log(0.99*EDLP_Sol[5][16])*0.675334228*FLAG_Sol[5][16] - 0.006787335804765123*(1 - FLAG_Sol[5][16]))*exp(0.3541748555565*(1 - FLAG_Sol[0][16])*(1 - FLAG_Sol[5][16]) + 1.99*log(0.99*EDLP_Sol[5][16])*0.179775065*(1 - FLAG_Sol[0][16])*FLAG_Sol[5][16] + log(1.99*EDLP_Sol[0][16])*0.99*0.179775065*FLAG_Sol[0][16]*(1 - FLAG_Sol[5][16]) + log(1.99*EDLP_Sol[0][16])*log(0.99*EDLP_Sol[5][16])*0.179775065*FLAG_Sol[0][16]*FLAG_Sol[5][16]))*FLAG_Sol[0][16] + (1.99 - (1.99*(1 - TPR_Sol[0][17])*(1 - FLAG_Sol[0][17]) + 1.99*EDLP_Sol[0][17]*FLAG_Sol[0][17]))*(exp(log(1.99*EDLP_Sol[0][17])*-2.703521163*FLAG_Sol[0][17] - 1.8603865588172197*(1 - FLAG_Sol[0][17]) + 10.56492735)*exp(log(0.99*EDLP_Sol[5][17])*0.675334228*FLAG_Sol[5][17] - 0.006787335804765123*(1 - FLAG_Sol[5][17]))*exp(0.3541748555565*(1 - FLAG_Sol[0][17])*(1 - FLAG_Sol[5][17]) + 1.99*log(0.99*EDLP_Sol[5][17])*0.179775065*(1 - FLAG_Sol[0][17])*FLAG_Sol[5][17] + log(1.99*EDLP_Sol[0][17])*0.99*0.179775065*FLAG_Sol[0][17]*(1 - FLAG_Sol[5][17]) + log(1.99*EDLP_Sol[0][17])*log(0.99*EDLP_Sol[5][17])*0.179775065*FLAG_Sol[0][17]*FLAG_Sol[5][17]))*FLAG_Sol[0][17] + (1.99 - (1.99*(1 - TPR_Sol[0][18])*(1 - FLAG_Sol[0][18]) + 1.99*EDLP_Sol[0][18]*FLAG_Sol[0][18]))*(exp(log(1.99*EDLP_Sol[0][18])*-2.703521163*FLAG_Sol[0][18] - 1.8603865588172197*(1 - FLAG_Sol[0][18]) + 10.5577298)*exp(log(0.99*EDLP_Sol[5][18])*0.675334228*FLAG_Sol[5][18] - 0.006787335804765123*(1 - FLAG_Sol[5][18]))*exp(0.3541748555565*(1 - FLAG_Sol[0][18])*(1 - FLAG_Sol[5][18]) + 1.99*log(0.99*EDLP_Sol[5][18])*0.179775065*(1 - FLAG_Sol[0][18])*FLAG_Sol[5][18] + log(1.99*EDLP_Sol[0][18])*0.99*0.179775065*FLAG_Sol[0][18]*(1 - FLAG_Sol[5][18]) + log(1.99*EDLP_Sol[0][18])*log(0.99*EDLP_Sol[5][18])*0.179775065*FLAG_Sol[0][18]*FLAG_Sol[5][18]))*FLAG_Sol[0][18] + (1.99 - (1.99*(1 - TPR_Sol[0][19])*(1 - FLAG_Sol[0][19]) + 1.99*EDLP_Sol[0][19]*FLAG_Sol[0][19]))*(exp(log(1.99*EDLP_Sol[0][19])*-2.703521163*FLAG_Sol[0][19] - 1.8603865588172197*(1 - FLAG_Sol[0][19]) + 10.54829571)*exp(log(0.99*EDLP_Sol[5][19])*0.675334228*FLAG_Sol[5][19] - 0.006787335804765123*(1 - FLAG_Sol[5][19]))*exp(0.3541748555565*(1 - FLAG_Sol[0][19])*(1 - FLAG_Sol[5][19]) + 1.99*log(0.99*EDLP_Sol[5][19])*0.179775065*(1 - FLAG_Sol[0][19])*FLAG_Sol[5][19] + log(1.99*EDLP_Sol[0][19])*0.99*0.179775065*FLAG_Sol[0][19]*(1 - FLAG_Sol[5][19]) + log(1.99*EDLP_Sol[0][19])*log(0.99*EDLP_Sol[5][19])*0.179775065*FLAG_Sol[0][19]*FLAG_Sol[5][19]))*FLAG_Sol[0][19] + (1.99 - (1.99*(1 - TPR_Sol[0][20])*(1 - FLAG_Sol[0][20]) + 1.99*EDLP_Sol[0][20]*FLAG_Sol[0][20]))*(exp(log(1.99*EDLP_Sol[0][20])*-2.703521163*FLAG_Sol[0][20] - 1.8603865588172197*(1 - FLAG_Sol[0][20]) + 10.55008055)*exp(log(0.99*EDLP_Sol[5][20])*0.675334228*FLAG_Sol[5][20] - 0.006787335804765123*(1 - FLAG_Sol[5][20]))*exp(0.3541748555565*(1 - FLAG_Sol[0][20])*(1 - FLAG_Sol[5][20]) + 1.99*log(0.99*EDLP_Sol[5][20])*0.179775065*(1 - FLAG_Sol[0][20])*FLAG_Sol[5][20] + log(1.99*EDLP_Sol[0][20])*0.99*0.179775065*FLAG_Sol[0][20]*(1 - FLAG_Sol[5][20]) + log(1.99*EDLP_Sol[0][20])*log(0.99*EDLP_Sol[5][20])*0.179775065*FLAG_Sol[0][20]*FLAG_Sol[5][20]))*FLAG_Sol[0][20] + (1.99 - (1.99*(1 - TPR_Sol[0][21])*(1 - FLAG_Sol[0][21]) + 1.99*EDLP_Sol[0][21]*FLAG_Sol[0][21]))*(exp(log(1.99*EDLP_Sol[0][21])*-2.703521163*FLAG_Sol[0][21] - 1.8603865588172197*(1 - FLAG_Sol[0][21]) + 10.54193597)*exp(log(0.99*EDLP_Sol[5][21])*0.675334228*FLAG_Sol[5][21] - 0.006787335804765123*(1 - FLAG_Sol[5][21]))*exp(0.3541748555565*(1 - FLAG_Sol[0][21])*(1 - FLAG_Sol[5][21]) + 1.99*log(0.99*EDLP_Sol[5][21])*0.179775065*(1 - FLAG_Sol[0][21])*FLAG_Sol[5][21] + log(1.99*EDLP_Sol[0][21])*0.99*0.179775065*FLAG_Sol[0][21]*(1 - FLAG_Sol[5][21]) + log(1.99*EDLP_Sol[0][21])*log(0.99*EDLP_Sol[5][21])*0.179775065*FLAG_Sol[0][21]*FLAG_Sol[5][21]))*FLAG_Sol[0][21] + (1.99 - (1.99*(1 - TPR_Sol[0][22])*(1 - FLAG_Sol[0][22]) + 1.99*EDLP_Sol[0][22]*FLAG_Sol[0][22]))*(exp(log(1.99*EDLP_Sol[0][22])*-2.703521163*FLAG_Sol[0][22] - 1.8603865588172197*(1 - FLAG_Sol[0][22]) + 10.55177377)*exp(log(0.99*EDLP_Sol[5][22])*0.675334228*FLAG_Sol[5][22] - 0.006787335804765123*(1 - FLAG_Sol[5][22]))*exp(0.3541748555565*(1 - FLAG_Sol[0][22])*(1 - FLAG_Sol[5][22]) + 1.99*log(0.99*EDLP_Sol[5][22])*0.179775065*(1 - FLAG_Sol[0][22])*FLAG_Sol[5][22] + log(1.99*EDLP_Sol[0][22])*0.99*0.179775065*FLAG_Sol[0][22]*(1 - FLAG_Sol[5][22]) + log(1.99*EDLP_Sol[0][22])*log(0.99*EDLP_Sol[5][22])*0.179775065*FLAG_Sol[0][22]*FLAG_Sol[5][22]))*FLAG_Sol[0][22] + (1.99 - (1.99*(1 - TPR_Sol[0][23])*(1 - FLAG_Sol[0][23]) + 1.99*EDLP_Sol[0][23]*FLAG_Sol[0][23]))*(exp(log(1.99*EDLP_Sol[0][23])*-2.703521163*FLAG_Sol[0][23] - 1.8603865588172197*(1 - FLAG_Sol[0][23]) + 10.54824624)*exp(log(0.99*EDLP_Sol[5][23])*0.675334228*FLAG_Sol[5][23] - 0.006787335804765123*(1 - FLAG_Sol[5][23]))*exp(0.3541748555565*(1 - FLAG_Sol[0][23])*(1 - FLAG_Sol[5][23]) + 1.99*log(0.99*EDLP_Sol[5][23])*0.179775065*(1 - FLAG_Sol[0][23])*FLAG_Sol[5][23] + log(1.99*EDLP_Sol[0][23])*0.99*0.179775065*FLAG_Sol[0][23]*(1 - FLAG_Sol[5][23]) + log(1.99*EDLP_Sol[0][23])*log(0.99*EDLP_Sol[5][23])*0.179775065*FLAG_Sol[0][23]*FLAG_Sol[5][23]))*FLAG_Sol[0][23] + (1.99 - (1.99*(1 - TPR_Sol[0][24])*(1 - FLAG_Sol[0][24]) + 1.99*EDLP_Sol[0][24]*FLAG_Sol[0][24]))*(exp(log(1.99*EDLP_Sol[0][24])*-2.703521163*FLAG_Sol[0][24] - 1.8603865588172197*(1 - FLAG_Sol[0][24]) + 10.55014121)*exp(log(0.99*EDLP_Sol[5][24])*0.675334228*FLAG_Sol[5][24] - 0.006787335804765123*(1 - FLAG_Sol[5][24]))*exp(0.3541748555565*(1 - FLAG_Sol[0][24])*(1 - FLAG_Sol[5][24]) + 1.99*log(0.99*EDLP_Sol[5][24])*0.179775065*(1 - FLAG_Sol[0][24])*FLAG_Sol[5][24] + log(1.99*EDLP_Sol[0][24])*0.99*0.179775065*FLAG_Sol[0][24]*(1 - FLAG_Sol[5][24]) + log(1.99*EDLP_Sol[0][24])*log(0.99*EDLP_Sol[5][24])*0.179775065*FLAG_Sol[0][24]*FLAG_Sol[5][24]))*FLAG_Sol[0][24] + (1.99 - (1.99*(1 - TPR_Sol[0][25])*(1 - FLAG_Sol[0][25]) + 1.99*EDLP_Sol[0][25]*FLAG_Sol[0][25]))*(exp(log(1.99*EDLP_Sol[0][25])*-2.703521163*FLAG_Sol[0][25] - 1.8603865588172197*(1 - FLAG_Sol[0][25]) + 10.548061599999999)*exp(log(0.99*EDLP_Sol[5][25])*0.675334228*FLAG_Sol[5][25] - 0.006787335804765123*(1 - FLAG_Sol[5][25]))*exp(0.3541748555565*(1 - FLAG_Sol[0][25])*(1 - FLAG_Sol[5][25]) + 1.99*log(0.99*EDLP_Sol[5][25])*0.179775065*(1 - FLAG_Sol[0][25])*FLAG_Sol[5][25] + log(1.99*EDLP_Sol[0][25])*0.99*0.179775065*FLAG_Sol[0][25]*(1 - FLAG_Sol[5][25]) + log(1.99*EDLP_Sol[0][25])*log(0.99*EDLP_Sol[5][25])*0.179775065*FLAG_Sol[0][25]*FLAG_Sol[5][25]))*FLAG_Sol[0][25] + (1.99 - (1.99*(1 - TPR_Sol[0][26])*(1 - FLAG_Sol[0][26]) + 1.99*EDLP_Sol[0][26]*FLAG_Sol[0][26]))*(exp(log(1.99*EDLP_Sol[0][26])*-2.703521163*FLAG_Sol[0][26] - 1.8603865588172197*(1 - FLAG_Sol[0][26]) + 10.56055207)*exp(log(0.99*EDLP_Sol[5][26])*0.675334228*FLAG_Sol[5][26] - 0.006787335804765123*(1 - FLAG_Sol[5][26]))*exp(0.3541748555565*(1 - FLAG_Sol[0][26])*(1 - FLAG_Sol[5][26]) + 1.99*log(0.99*EDLP_Sol[5][26])*0.179775065*(1 - FLAG_Sol[0][26])*FLAG_Sol[5][26] + log(1.99*EDLP_Sol[0][26])*0.99*0.179775065*FLAG_Sol[0][26]*(1 - FLAG_Sol[5][26]) + log(1.99*EDLP_Sol[0][26])*log(0.99*EDLP_Sol[5][26])*0.179775065*FLAG_Sol[0][26]*FLAG_Sol[5][26]))*FLAG_Sol[0][26] + (1.99 - (1.99*(1 - TPR_Sol[0][27])*(1 - FLAG_Sol[0][27]) + 1.99*EDLP_Sol[0][27]*FLAG_Sol[0][27]))*(exp(log(1.99*EDLP_Sol[0][27])*-2.703521163*FLAG_Sol[0][27] - 1.8603865588172197*(1 - FLAG_Sol[0][27]) + 10.54280649)*exp(log(0.99*EDLP_Sol[5][27])*0.675334228*FLAG_Sol[5][27] - 0.006787335804765123*(1 - FLAG_Sol[5][27]))*exp(0.3541748555565*(1 - FLAG_Sol[0][27])*(1 - FLAG_Sol[5][27]) + 1.99*log(0.99*EDLP_Sol[5][27])*0.179775065*(1 - FLAG_Sol[0][27])*FLAG_Sol[5][27] + log(1.99*EDLP_Sol[0][27])*0.99*0.179775065*FLAG_Sol[0][27]*(1 - FLAG_Sol[5][27]) + log(1.99*EDLP_Sol[0][27])*log(0.99*EDLP_Sol[5][27])*0.179775065*FLAG_Sol[0][27]*FLAG_Sol[5][27]))*FLAG_Sol[0][27] + (1.99 - (1.99*(1 - TPR_Sol[0][28])*(1 - FLAG_Sol[0][28]) + 1.99*EDLP_Sol[0][28]*FLAG_Sol[0][28]))*(exp(log(1.99*EDLP_Sol[0][28])*-2.703521163*FLAG_Sol[0][28] - 1.8603865588172197*(1 - FLAG_Sol[0][28]) + 10.54481865)*exp(log(0.99*EDLP_Sol[5][28])*0.675334228*FLAG_Sol[5][28] - 0.006787335804765123*(1 - FLAG_Sol[5][28]))*exp(0.3541748555565*(1 - FLAG_Sol[0][28])*(1 - FLAG_Sol[5][28]) + 1.99*log(0.99*EDLP_Sol[5][28])*0.179775065*(1 - FLAG_Sol[0][28])*FLAG_Sol[5][28] + log(1.99*EDLP_Sol[0][28])*0.99*0.179775065*FLAG_Sol[0][28]*(1 - FLAG_Sol[5][28]) + log(1.99*EDLP_Sol[0][28])*log(0.99*EDLP_Sol[5][28])*0.179775065*FLAG_Sol[0][28]*FLAG_Sol[5][28]))*FLAG_Sol[0][28] + (1.99 - (1.99*(1 - TPR_Sol[0][29])*(1 - FLAG_Sol[0][29]) + 1.99*EDLP_Sol[0][29]*FLAG_Sol[0][29]))*(exp(log(1.99*EDLP_Sol[0][29])*-2.703521163*FLAG_Sol[0][29] - 1.8603865588172197*(1 - FLAG_Sol[0][29]) + 10.54362816)*exp(log(0.99*EDLP_Sol[5][29])*0.675334228*FLAG_Sol[5][29] - 0.006787335804765123*(1 - FLAG_Sol[5][29]))*exp(0.3541748555565*(1 - FLAG_Sol[0][29])*(1 - FLAG_Sol[5][29]) + 1.99*log(0.99*EDLP_Sol[5][29])*0.179775065*(1 - FLAG_Sol[0][29])*FLAG_Sol[5][29] + log(1.99*EDLP_Sol[0][29])*0.99*0.179775065*FLAG_Sol[0][29]*(1 - FLAG_Sol[5][29]) + log(1.99*EDLP_Sol[0][29])*log(0.99*EDLP_Sol[5][29])*0.179775065*FLAG_Sol[0][29]*FLAG_Sol[5][29]))*FLAG_Sol[0][29] + (1.99 - (1.99*(1 - TPR_Sol[0][30])*(1 - FLAG_Sol[0][30]) + 1.99*EDLP_Sol[0][30]*FLAG_Sol[0][30]))*(exp(log(1.99*EDLP_Sol[0][30])*-2.703521163*FLAG_Sol[0][30] - 1.8603865588172197*(1 - FLAG_Sol[0][30]) + 10.55220403)*exp(log(0.99*EDLP_Sol[5][30])*0.675334228*FLAG_Sol[5][30] - 0.006787335804765123*(1 - FLAG_Sol[5][30]))*exp(0.3541748555565*(1 - FLAG_Sol[0][30])*(1 - FLAG_Sol[5][30]) + 1.99*log(0.99*EDLP_Sol[5][30])*0.179775065*(1 - FLAG_Sol[0][30])*FLAG_Sol[5][30] + log(1.99*EDLP_Sol[0][30])*0.99*0.179775065*FLAG_Sol[0][30]*(1 - FLAG_Sol[5][30]) + log(1.99*EDLP_Sol[0][30])*log(0.99*EDLP_Sol[5][30])*0.179775065*FLAG_Sol[0][30]*FLAG_Sol[5][30]))*FLAG_Sol[0][30] + (1.99 - (1.99*(1 - TPR_Sol[0][31])*(1 - FLAG_Sol[0][31]) + 1.99*EDLP_Sol[0][31]*FLAG_Sol[0][31]))*(exp(log(1.99*EDLP_Sol[0][31])*-2.703521163*FLAG_Sol[0][31] - 1.8603865588172197*(1 - FLAG_Sol[0][31]) + 10.54552009)*exp(log(0.99*EDLP_Sol[5][31])*0.675334228*FLAG_Sol[5][31] - 0.006787335804765123*(1 - FLAG_Sol[5][31]))*exp(0.3541748555565*(1 - FLAG_Sol[0][31])*(1 - FLAG_Sol[5][31]) + 1.99*log(0.99*EDLP_Sol[5][31])*0.179775065*(1 - FLAG_Sol[0][31])*FLAG_Sol[5][31] + log(1.99*EDLP_Sol[0][31])*0.99*0.179775065*FLAG_Sol[0][31]*(1 - FLAG_Sol[5][31]) + log(1.99*EDLP_Sol[0][31])*log(0.99*EDLP_Sol[5][31])*0.179775065*FLAG_Sol[0][31]*FLAG_Sol[5][31]))*FLAG_Sol[0][31] + (1.99 - (1.99*(1 - TPR_Sol[0][32])*(1 - FLAG_Sol[0][32]) + 1.99*EDLP_Sol[0][32]*FLAG_Sol[0][32]))*(exp(log(1.99*EDLP_Sol[0][32])*-2.703521163*FLAG_Sol[0][32] - 1.8603865588172197*(1 - FLAG_Sol[0][32]) + 10.546682)*exp(log(0.99*EDLP_Sol[5][32])*0.675334228*FLAG_Sol[5][32] - 0.006787335804765123*(1 - FLAG_Sol[5][32]))*exp(0.3541748555565*(1 - FLAG_Sol[0][32])*(1 - FLAG_Sol[5][32]) + 1.99*log(0.99*EDLP_Sol[5][32])*0.179775065*(1 - FLAG_Sol[0][32])*FLAG_Sol[5][32] + log(1.99*EDLP_Sol[0][32])*0.99*0.179775065*FLAG_Sol[0][32]*(1 - FLAG_Sol[5][32]) + log(1.99*EDLP_Sol[0][32])*log(0.99*EDLP_Sol[5][32])*0.179775065*FLAG_Sol[0][32]*FLAG_Sol[5][32]))*FLAG_Sol[0][32] + (1.99 - (1.99*(1 - TPR_Sol[0][33])*(1 - FLAG_Sol[0][33]) + 1.99*EDLP_Sol[0][33]*FLAG_Sol[0][33]))*(exp(log(1.99*EDLP_Sol[0][33])*-2.703521163*FLAG_Sol[0][33] - 1.8603865588172197*(1 - FLAG_Sol[0][33]) + 10.54644992)*exp(log(0.99*EDLP_Sol[5][33])*0.675334228*FLAG_Sol[5][33] - 0.006787335804765123*(1 - FLAG_Sol[5][33]))*exp(0.3541748555565*(1 - FLAG_Sol[0][33])*(1 - FLAG_Sol[5][33]) + 1.99*log(0.99*EDLP_Sol[5][33])*0.179775065*(1 - FLAG_Sol[0][33])*FLAG_Sol[5][33] + log(1.99*EDLP_Sol[0][33])*0.99*0.179775065*FLAG_Sol[0][33]*(1 - FLAG_Sol[5][33]) + log(1.99*EDLP_Sol[0][33])*log(0.99*EDLP_Sol[5][33])*0.179775065*FLAG_Sol[0][33]*FLAG_Sol[5][33]))*FLAG_Sol[0][33] + (1.99 - (1.99*(1 - TPR_Sol[0][34])*(1 - FLAG_Sol[0][34]) + 1.99*EDLP_Sol[0][34]*FLAG_Sol[0][34]))*(exp(log(1.99*EDLP_Sol[0][34])*-2.703521163*FLAG_Sol[0][34] - 1.8603865588172197*(1 - FLAG_Sol[0][34]) + 10.55107867)*exp(log(0.99*EDLP_Sol[5][34])*0.675334228*FLAG_Sol[5][34] - 0.006787335804765123*(1 - FLAG_Sol[5][34]))*exp(0.3541748555565*(1 - FLAG_Sol[0][34])*(1 - FLAG_Sol[5][34]) + 1.99*log(0.99*EDLP_Sol[5][34])*0.179775065*(1 - FLAG_Sol[0][34])*FLAG_Sol[5][34] + log(1.99*EDLP_Sol[0][34])*0.99*0.179775065*FLAG_Sol[0][34]*(1 - FLAG_Sol[5][34]) + log(1.99*EDLP_Sol[0][34])*log(0.99*EDLP_Sol[5][34])*0.179775065*FLAG_Sol[0][34]*FLAG_Sol[5][34]))*FLAG_Sol[0][34] + (1.99 - (1.99*(1 - TPR_Sol[0][35])*(1 - FLAG_Sol[0][35]) + 1.99*EDLP_Sol[0][35]*FLAG_Sol[0][35]))*(exp(log(1.99*EDLP_Sol[0][35])*-2.703521163*FLAG_Sol[0][35] - 1.8603865588172197*(1 - FLAG_Sol[0][35]) + 10.55170651)*exp(log(0.99*EDLP_Sol[5][35])*0.675334228*FLAG_Sol[5][35] - 0.006787335804765123*(1 - FLAG_Sol[5][35]))*exp(0.3541748555565*(1 - FLAG_Sol[0][35])*(1 - FLAG_Sol[5][35]) + 1.99*log(0.99*EDLP_Sol[5][35])*0.179775065*(1 - FLAG_Sol[0][35])*FLAG_Sol[5][35] + log(1.99*EDLP_Sol[0][35])*0.99*0.179775065*FLAG_Sol[0][35]*(1 - FLAG_Sol[5][35]) + log(1.99*EDLP_Sol[0][35])*log(0.99*EDLP_Sol[5][35])*0.179775065*FLAG_Sol[0][35]*FLAG_Sol[5][35]))*FLAG_Sol[0][35] + (1.99 - (1.99*(1 - TPR_Sol[0][36])*(1 - FLAG_Sol[0][36]) + 1.99*EDLP_Sol[0][36]*FLAG_Sol[0][36]))*(exp(log(1.99*EDLP_Sol[0][36])*-2.703521163*FLAG_Sol[0][36] - 1.8603865588172197*(1 - FLAG_Sol[0][36]) + 10.53993193)*exp(log(0.99*EDLP_Sol[5][36])*0.675334228*FLAG_Sol[5][36] - 0.006787335804765123*(1 - FLAG_Sol[5][36]))*exp(0.3541748555565*(1 - FLAG_Sol[0][36])*(1 - FLAG_Sol[5][36]) + 1.99*log(0.99*EDLP_Sol[5][36])*0.179775065*(1 - FLAG_Sol[0][36])*FLAG_Sol[5][36] + log(1.99*EDLP_Sol[0][36])*0.99*0.179775065*FLAG_Sol[0][36]*(1 - FLAG_Sol[5][36]) + log(1.99*EDLP_Sol[0][36])*log(0.99*EDLP_Sol[5][36])*0.179775065*FLAG_Sol[0][36]*FLAG_Sol[5][36]))*FLAG_Sol[0][36] + (1.99 - (1.99*(1 - TPR_Sol[0][37])*(1 - FLAG_Sol[0][37]) + 1.99*EDLP_Sol[0][37]*FLAG_Sol[0][37]))*(exp(log(1.99*EDLP_Sol[0][37])*-2.703521163*FLAG_Sol[0][37] - 1.8603865588172197*(1 - FLAG_Sol[0][37]) + 10.54254286)*exp(log(0.99*EDLP_Sol[5][37])*0.675334228*FLAG_Sol[5][37] - 0.006787335804765123*(1 - FLAG_Sol[5][37]))*exp(0.3541748555565*(1 - FLAG_Sol[0][37])*(1 - FLAG_Sol[5][37]) + 1.99*log(0.99*EDLP_Sol[5][37])*0.179775065*(1 - FLAG_Sol[0][37])*FLAG_Sol[5][37] + log(1.99*EDLP_Sol[0][37])*0.99*0.179775065*FLAG_Sol[0][37]*(1 - FLAG_Sol[5][37]) + log(1.99*EDLP_Sol[0][37])*log(0.99*EDLP_Sol[5][37])*0.179775065*FLAG_Sol[0][37]*FLAG_Sol[5][37]))*FLAG_Sol[0][37] + (1.99 - (1.99*(1 - TPR_Sol[0][38])*(1 - FLAG_Sol[0][38]) + 1.99*EDLP_Sol[0][38]*FLAG_Sol[0][38]))*(exp(log(1.99*EDLP_Sol[0][38])*-2.703521163*FLAG_Sol[0][38] - 1.8603865588172197*(1 - FLAG_Sol[0][38]) + 10.54262426)*exp(log(0.99*EDLP_Sol[5][38])*0.675334228*FLAG_Sol[5][38] - 0.006787335804765123*(1 - FLAG_Sol[5][38]))*exp(0.3541748555565*(1 - FLAG_Sol[0][38])*(1 - FLAG_Sol[5][38]) + 1.99*log(0.99*EDLP_Sol[5][38])*0.179775065*(1 - FLAG_Sol[0][38])*FLAG_Sol[5][38] + log(1.99*EDLP_Sol[0][38])*0.99*0.179775065*FLAG_Sol[0][38]*(1 - FLAG_Sol[5][38]) + log(1.99*EDLP_Sol[0][38])*log(0.99*EDLP_Sol[5][38])*0.179775065*FLAG_Sol[0][38]*FLAG_Sol[5][38]))*FLAG_Sol[0][38] + (1.99 - (1.99*(1 - TPR_Sol[0][39])*(1 - FLAG_Sol[0][39]) + 1.99*EDLP_Sol[0][39]*FLAG_Sol[0][39]))*(exp(log(1.99*EDLP_Sol[0][39])*-2.703521163*FLAG_Sol[0][39] - 1.8603865588172197*(1 - FLAG_Sol[0][39]) + 10.55587125)*exp(log(0.99*EDLP_Sol[5][39])*0.675334228*FLAG_Sol[5][39] - 0.006787335804765123*(1 - FLAG_Sol[5][39]))*exp(0.3541748555565*(1 - FLAG_Sol[0][39])*(1 - FLAG_Sol[5][39]) + 1.99*log(0.99*EDLP_Sol[5][39])*0.179775065*(1 - FLAG_Sol[0][39])*FLAG_Sol[5][39] + log(1.99*EDLP_Sol[0][39])*0.99*0.179775065*FLAG_Sol[0][39]*(1 - FLAG_Sol[5][39]) + log(1.99*EDLP_Sol[0][39])*log(0.99*EDLP_Sol[5][39])*0.179775065*FLAG_Sol[0][39]*FLAG_Sol[5][39]))*FLAG_Sol[0][39] + (1.99 - (1.99*(1 - TPR_Sol[0][40])*(1 - FLAG_Sol[0][40]) + 1.99*EDLP_Sol[0][40]*FLAG_Sol[0][40]))*(exp(log(1.99*EDLP_Sol[0][40])*-2.703521163*FLAG_Sol[0][40] - 1.8603865588172197*(1 - FLAG_Sol[0][40]) + 10.55370541)*exp(log(0.99*EDLP_Sol[5][40])*0.675334228*FLAG_Sol[5][40] - 0.006787335804765123*(1 - FLAG_Sol[5][40]))*exp(0.3541748555565*(1 - FLAG_Sol[0][40])*(1 - FLAG_Sol[5][40]) + 1.99*log(0.99*EDLP_Sol[5][40])*0.179775065*(1 - FLAG_Sol[0][40])*FLAG_Sol[5][40] + log(1.99*EDLP_Sol[0][40])*0.99*0.179775065*FLAG_Sol[0][40]*(1 - FLAG_Sol[5][40]) + log(1.99*EDLP_Sol[0][40])*log(0.99*EDLP_Sol[5][40])*0.179775065*FLAG_Sol[0][40]*FLAG_Sol[5][40]))*FLAG_Sol[0][40] + (1.99 - (1.99*(1 - TPR_Sol[0][41])*(1 - FLAG_Sol[0][41]) + 1.99*EDLP_Sol[0][41]*FLAG_Sol[0][41]))*(exp(log(1.99*EDLP_Sol[0][41])*-2.703521163*FLAG_Sol[0][41] - 1.8603865588172197*(1 - FLAG_Sol[0][41]) + 10.55256282)*exp(log(0.99*EDLP_Sol[5][41])*0.675334228*FLAG_Sol[5][41] - 0.006787335804765123*(1 - FLAG_Sol[5][41]))*exp(0.3541748555565*(1 - FLAG_Sol[0][41])*(1 - FLAG_Sol[5][41]) + 1.99*log(0.99*EDLP_Sol[5][41])*0.179775065*(1 - FLAG_Sol[0][41])*FLAG_Sol[5][41] + log(1.99*EDLP_Sol[0][41])*0.99*0.179775065*FLAG_Sol[0][41]*(1 - FLAG_Sol[5][41]) + log(1.99*EDLP_Sol[0][41])*log(0.99*EDLP_Sol[5][41])*0.179775065*FLAG_Sol[0][41]*FLAG_Sol[5][41]))*FLAG_Sol[0][41] + (1.99 - (1.99*(1 - TPR_Sol[0][42])*(1 - FLAG_Sol[0][42]) + 1.99*EDLP_Sol[0][42]*FLAG_Sol[0][42]))*(exp(log(1.99*EDLP_Sol[0][42])*-2.703521163*FLAG_Sol[0][42] - 1.8603865588172197*(1 - FLAG_Sol[0][42]) + 10.55192396)*exp(log(0.99*EDLP_Sol[5][42])*0.675334228*FLAG_Sol[5][42] - 0.006787335804765123*(1 - FLAG_Sol[5][42]))*exp(0.3541748555565*(1 - FLAG_Sol[0][42])*(1 - FLAG_Sol[5][42]) + 1.99*log(0.99*EDLP_Sol[5][42])*0.179775065*(1 - FLAG_Sol[0][42])*FLAG_Sol[5][42] + log(1.99*EDLP_Sol[0][42])*0.99*0.179775065*FLAG_Sol[0][42]*(1 - FLAG_Sol[5][42]) + log(1.99*EDLP_Sol[0][42])*log(0.99*EDLP_Sol[5][42])*0.179775065*FLAG_Sol[0][42]*FLAG_Sol[5][42]))*FLAG_Sol[0][42] + (1.99 - (1.99*(1 - TPR_Sol[0][43])*(1 - FLAG_Sol[0][43]) + 1.99*EDLP_Sol[0][43]*FLAG_Sol[0][43]))*(exp(log(1.99*EDLP_Sol[0][43])*-2.703521163*FLAG_Sol[0][43] - 1.8603865588172197*(1 - FLAG_Sol[0][43]) + 10.56130311)*exp(log(0.99*EDLP_Sol[5][43])*0.675334228*FLAG_Sol[5][43] - 0.006787335804765123*(1 - FLAG_Sol[5][43]))*exp(0.3541748555565*(1 - FLAG_Sol[0][43])*(1 - FLAG_Sol[5][43]) + 1.99*log(0.99*EDLP_Sol[5][43])*0.179775065*(1 - FLAG_Sol[0][43])*FLAG_Sol[5][43] + log(1.99*EDLP_Sol[0][43])*0.99*0.179775065*FLAG_Sol[0][43]*(1 - FLAG_Sol[5][43]) + log(1.99*EDLP_Sol[0][43])*log(0.99*EDLP_Sol[5][43])*0.179775065*FLAG_Sol[0][43]*FLAG_Sol[5][43]))*FLAG_Sol[0][43] + (1.99 - (1.99*(1 - TPR_Sol[0][44])*(1 - FLAG_Sol[0][44]) + 1.99*EDLP_Sol[0][44]*FLAG_Sol[0][44]))*(exp(log(1.99*EDLP_Sol[0][44])*-2.703521163*FLAG_Sol[0][44] - 1.8603865588172197*(1 - FLAG_Sol[0][44]) + 10.56211031)*exp(log(0.99*EDLP_Sol[5][44])*0.675334228*FLAG_Sol[5][44] - 0.006787335804765123*(1 - FLAG_Sol[5][44]))*exp(0.3541748555565*(1 - FLAG_Sol[0][44])*(1 - FLAG_Sol[5][44]) + 1.99*log(0.99*EDLP_Sol[5][44])*0.179775065*(1 - FLAG_Sol[0][44])*FLAG_Sol[5][44] + log(1.99*EDLP_Sol[0][44])*0.99*0.179775065*FLAG_Sol[0][44]*(1 - FLAG_Sol[5][44]) + log(1.99*EDLP_Sol[0][44])*log(0.99*EDLP_Sol[5][44])*0.179775065*FLAG_Sol[0][44]*FLAG_Sol[5][44]))*FLAG_Sol[0][44] + (1.99 - (1.99*(1 - TPR_Sol[0][45])*(1 - FLAG_Sol[0][45]) + 1.99*EDLP_Sol[0][45]*FLAG_Sol[0][45]))*(exp(log(1.99*EDLP_Sol[0][45])*-2.703521163*FLAG_Sol[0][45] - 1.8603865588172197*(1 - FLAG_Sol[0][45]) + 10.55738633)*exp(log(0.99*EDLP_Sol[5][45])*0.675334228*FLAG_Sol[5][45] - 0.006787335804765123*(1 - FLAG_Sol[5][45]))*exp(0.3541748555565*(1 - FLAG_Sol[0][45])*(1 - FLAG_Sol[5][45]) + 1.99*log(0.99*EDLP_Sol[5][45])*0.179775065*(1 - FLAG_Sol[0][45])*FLAG_Sol[5][45] + log(1.99*EDLP_Sol[0][45])*0.99*0.179775065*FLAG_Sol[0][45]*(1 - FLAG_Sol[5][45]) + log(1.99*EDLP_Sol[0][45])*log(0.99*EDLP_Sol[5][45])*0.179775065*FLAG_Sol[0][45]*FLAG_Sol[5][45]))*FLAG_Sol[0][45] + (1.99 - (1.99*(1 - TPR_Sol[0][46])*(1 - FLAG_Sol[0][46]) + 1.99*EDLP_Sol[0][46]*FLAG_Sol[0][46]))*(exp(log(1.99*EDLP_Sol[0][46])*-2.703521163*FLAG_Sol[0][46] - 1.8603865588172197*(1 - FLAG_Sol[0][46]) + 10.56232061)*exp(log(0.99*EDLP_Sol[5][46])*0.675334228*FLAG_Sol[5][46] - 0.006787335804765123*(1 - FLAG_Sol[5][46]))*exp(0.3541748555565*(1 - FLAG_Sol[0][46])*(1 - FLAG_Sol[5][46]) + 1.99*log(0.99*EDLP_Sol[5][46])*0.179775065*(1 - FLAG_Sol[0][46])*FLAG_Sol[5][46] + log(1.99*EDLP_Sol[0][46])*0.99*0.179775065*FLAG_Sol[0][46]*(1 - FLAG_Sol[5][46]) + log(1.99*EDLP_Sol[0][46])*log(0.99*EDLP_Sol[5][46])*0.179775065*FLAG_Sol[0][46]*FLAG_Sol[5][46]))*FLAG_Sol[0][46] + (1.99 - (1.99*(1 - TPR_Sol[0][47])*(1 - FLAG_Sol[0][47]) + 1.99*EDLP_Sol[0][47]*FLAG_Sol[0][47]))*(exp(log(1.99*EDLP_Sol[0][47])*-2.703521163*FLAG_Sol[0][47] - 1.8603865588172197*(1 - FLAG_Sol[0][47]) + 10.55758064)*exp(log(0.99*EDLP_Sol[5][47])*0.675334228*FLAG_Sol[5][47] - 0.006787335804765123*(1 - FLAG_Sol[5][47]))*exp(0.3541748555565*(1 - FLAG_Sol[0][47])*(1 - FLAG_Sol[5][47]) + 1.99*log(0.99*EDLP_Sol[5][47])*0.179775065*(1 - FLAG_Sol[0][47])*FLAG_Sol[5][47] + log(1.99*EDLP_Sol[0][47])*0.99*0.179775065*FLAG_Sol[0][47]*(1 - FLAG_Sol[5][47]) + log(1.99*EDLP_Sol[0][47])*log(0.99*EDLP_Sol[5][47])*0.179775065*FLAG_Sol[0][47]*FLAG_Sol[5][47]))*FLAG_Sol[0][47] + (1.99 - (1.99*(1 - TPR_Sol[0][48])*(1 - FLAG_Sol[0][48]) + 1.99*EDLP_Sol[0][48]*FLAG_Sol[0][48]))*(exp(log(1.99*EDLP_Sol[0][48])*-2.703521163*FLAG_Sol[0][48] - 1.8603865588172197*(1 - FLAG_Sol[0][48]) + 10.56182234)*exp(log(0.99*EDLP_Sol[5][48])*0.675334228*FLAG_Sol[5][48] - 0.006787335804765123*(1 - FLAG_Sol[5][48]))*exp(0.3541748555565*(1 - FLAG_Sol[0][48])*(1 - FLAG_Sol[5][48]) + 1.99*log(0.99*EDLP_Sol[5][48])*0.179775065*(1 - FLAG_Sol[0][48])*FLAG_Sol[5][48] + log(1.99*EDLP_Sol[0][48])*0.99*0.179775065*FLAG_Sol[0][48]*(1 - FLAG_Sol[5][48]) + log(1.99*EDLP_Sol[0][48])*log(0.99*EDLP_Sol[5][48])*0.179775065*FLAG_Sol[0][48]*FLAG_Sol[5][48]))*FLAG_Sol[0][48] + (1.99 - (1.99*(1 - TPR_Sol[0][49])*(1 - FLAG_Sol[0][49]) + 1.99*EDLP_Sol[0][49]*FLAG_Sol[0][49]))*(exp(log(1.99*EDLP_Sol[0][49])*-2.703521163*FLAG_Sol[0][49] - 1.8603865588172197*(1 - FLAG_Sol[0][49]) + 10.57289033)*exp(log(0.99*EDLP_Sol[5][49])*0.675334228*FLAG_Sol[5][49] - 0.006787335804765123*(1 - FLAG_Sol[5][49]))*exp(0.3541748555565*(1 - FLAG_Sol[0][49])*(1 - FLAG_Sol[5][49]) + 1.99*log(0.99*EDLP_Sol[5][49])*0.179775065*(1 - FLAG_Sol[0][49])*FLAG_Sol[5][49] + log(1.99*EDLP_Sol[0][49])*0.99*0.179775065*FLAG_Sol[0][49]*(1 - FLAG_Sol[5][49]) + log(1.99*EDLP_Sol[0][49])*log(0.99*EDLP_Sol[5][49])*0.179775065*FLAG_Sol[0][49]*FLAG_Sol[5][49]))*FLAG_Sol[0][49] + (1.99 - (1.99*(1 - TPR_Sol[0][50])*(1 - FLAG_Sol[0][50]) + 1.99*EDLP_Sol[0][50]*FLAG_Sol[0][50]))*(exp(log(1.99*EDLP_Sol[0][50])*-2.703521163*FLAG_Sol[0][50] - 1.8603865588172197*(1 - FLAG_Sol[0][50]) + 10.60062074)*exp(log(0.99*EDLP_Sol[5][50])*0.675334228*FLAG_Sol[5][50] - 0.006787335804765123*(1 - FLAG_Sol[5][50]))*exp(0.3541748555565*(1 - FLAG_Sol[0][50])*(1 - FLAG_Sol[5][50]) + 1.99*log(0.99*EDLP_Sol[5][50])*0.179775065*(1 - FLAG_Sol[0][50])*FLAG_Sol[5][50] + log(1.99*EDLP_Sol[0][50])*0.99*0.179775065*FLAG_Sol[0][50]*(1 - FLAG_Sol[5][50]) + log(1.99*EDLP_Sol[0][50])*log(0.99*EDLP_Sol[5][50])*0.179775065*FLAG_Sol[0][50]*FLAG_Sol[5][50]))*FLAG_Sol[0][50] + (1.99 - (1.99*(1 - TPR_Sol[0][51])*(1 - FLAG_Sol[0][51]) + 1.99*EDLP_Sol[0][51]*FLAG_Sol[0][51]))*(exp(log(1.99*EDLP_Sol[0][51])*-2.703521163*FLAG_Sol[0][51] - 1.8603865588172197*(1 - FLAG_Sol[0][51]) + 10.59679151)*exp(log(0.99*EDLP_Sol[5][51])*0.675334228*FLAG_Sol[5][51] - 0.006787335804765123*(1 - FLAG_Sol[5][51]))*exp(0.3541748555565*(1 - FLAG_Sol[0][51])*(1 - FLAG_Sol[5][51]) + 1.99*log(0.99*EDLP_Sol[5][51])*0.179775065*(1 - FLAG_Sol[0][51])*FLAG_Sol[5][51] + log(1.99*EDLP_Sol[0][51])*0.99*0.179775065*FLAG_Sol[0][51]*(1 - FLAG_Sol[5][51]) + log(1.99*EDLP_Sol[0][51])*log(0.99*EDLP_Sol[5][51])*0.179775065*FLAG_Sol[0][51]*FLAG_Sol[5][51]))*FLAG_Sol[0][51]

3137.737996124043

In [352]:
(3.45 - (3.45*(1 - TPR_Sol[4][0])*(1 - FLAG_Sol[4][0]) + 3.45*EDLP_Sol[4][0]*FLAG_Sol[4][0]))*exp(0.19395708600000003*TPR_Sol[4][0]*(1 - FLAG_Sol[4][0]) + 8.988413994)*FLAG_Sol[4][0] + (3.45 - (3.45*(1 - TPR_Sol[4][1])*(1 - FLAG_Sol[4][1]) + 3.45*EDLP_Sol[4][1]*FLAG_Sol[4][1]))*exp(0.19395708600000003*TPR_Sol[4][1]*(1 - FLAG_Sol[4][1]) + 8.988413994)*FLAG_Sol[4][1] + (3.45 - (3.45*(1 - TPR_Sol[4][2])*(1 - FLAG_Sol[4][2]) + 3.45*EDLP_Sol[4][2]*FLAG_Sol[4][2]))*exp(0.19395708600000003*TPR_Sol[4][2]*(1 - FLAG_Sol[4][2]) + 8.988413994)*FLAG_Sol[4][2] + (3.45 - (3.45*(1 - TPR_Sol[4][3])*(1 - FLAG_Sol[4][3]) + 3.45*EDLP_Sol[4][3]*FLAG_Sol[4][3]))*exp(0.19395708600000003*TPR_Sol[4][3]*(1 - FLAG_Sol[4][3]) + 8.987457015)*FLAG_Sol[4][3] + (3.45 - (3.45*(1 - TPR_Sol[4][4])*(1 - FLAG_Sol[4][4]) + 3.45*EDLP_Sol[4][4]*FLAG_Sol[4][4]))*exp(0.19395708600000003*TPR_Sol[4][4]*(1 - FLAG_Sol[4][4]) + 8.990367597999999)*FLAG_Sol[4][4] + (3.45 - (3.45*(1 - TPR_Sol[4][5])*(1 - FLAG_Sol[4][5]) + 3.45*EDLP_Sol[4][5]*FLAG_Sol[4][5]))*exp(0.19395708600000003*TPR_Sol[4][5]*(1 - FLAG_Sol[4][5]) + 8.9936849)*FLAG_Sol[4][5] + (3.45 - (3.45*(1 - TPR_Sol[4][6])*(1 - FLAG_Sol[4][6]) + 3.45*EDLP_Sol[4][6]*FLAG_Sol[4][6]))*exp(0.19395708600000003*TPR_Sol[4][6]*(1 - FLAG_Sol[4][6]) + 8.992921766)*FLAG_Sol[4][6] + (3.45 - (3.45*(1 - TPR_Sol[4][7])*(1 - FLAG_Sol[4][7]) + 3.45*EDLP_Sol[4][7]*FLAG_Sol[4][7]))*exp(0.19395708600000003*TPR_Sol[4][7]*(1 - FLAG_Sol[4][7]) + 8.995059989)*FLAG_Sol[4][7] + (3.45 - (3.45*(1 - TPR_Sol[4][8])*(1 - FLAG_Sol[4][8]) + 3.45*EDLP_Sol[4][8]*FLAG_Sol[4][8]))*exp(0.19395708600000003*TPR_Sol[4][8]*(1 - FLAG_Sol[4][8]) + 8.995287485)*FLAG_Sol[4][8] + (3.45 - (3.45*(1 - TPR_Sol[4][9])*(1 - FLAG_Sol[4][9]) + 3.45*EDLP_Sol[4][9]*FLAG_Sol[4][9]))*exp(0.19395708600000003*TPR_Sol[4][9]*(1 - FLAG_Sol[4][9]) + 8.995321188)*FLAG_Sol[4][9] + (3.45 - (3.45*(1 - TPR_Sol[4][10])*(1 - FLAG_Sol[4][10]) + 3.45*EDLP_Sol[4][10]*FLAG_Sol[4][10]))*exp(0.19395708600000003*TPR_Sol[4][10]*(1 - FLAG_Sol[4][10]) + 8.995661307)*FLAG_Sol[4][10] + (3.45 - (3.45*(1 - TPR_Sol[4][11])*(1 - FLAG_Sol[4][11]) + 3.45*EDLP_Sol[4][11]*FLAG_Sol[4][11]))*exp(0.19395708600000003*TPR_Sol[4][11]*(1 - FLAG_Sol[4][11]) + 8.995059989)*FLAG_Sol[4][11] + (3.45 - (3.45*(1 - TPR_Sol[4][12])*(1 - FLAG_Sol[4][12]) + 3.45*EDLP_Sol[4][12]*FLAG_Sol[4][12]))*exp(0.19395708600000003*TPR_Sol[4][12]*(1 - FLAG_Sol[4][12]) + 8.995059989)*FLAG_Sol[4][12] + (3.45 - (3.45*(1 - TPR_Sol[4][13])*(1 - FLAG_Sol[4][13]) + 3.45*EDLP_Sol[4][13]*FLAG_Sol[4][13]))*exp(0.19395708600000003*TPR_Sol[4][13]*(1 - FLAG_Sol[4][13]) + 8.996811888)*FLAG_Sol[4][13] + (3.45 - (3.45*(1 - TPR_Sol[4][14])*(1 - FLAG_Sol[4][14]) + 3.45*EDLP_Sol[4][14]*FLAG_Sol[4][14]))*exp(0.19395708600000003*TPR_Sol[4][14]*(1 - FLAG_Sol[4][14]) + 8.995287485)*FLAG_Sol[4][14] + (3.45 - (3.45*(1 - TPR_Sol[4][15])*(1 - FLAG_Sol[4][15]) + 3.45*EDLP_Sol[4][15]*FLAG_Sol[4][15]))*exp(0.19395708600000003*TPR_Sol[4][15]*(1 - FLAG_Sol[4][15]) + 8.995287485)*FLAG_Sol[4][15] + (3.45 - (3.45*(1 - TPR_Sol[4][16])*(1 - FLAG_Sol[4][16]) + 3.45*EDLP_Sol[4][16]*FLAG_Sol[4][16]))*exp(0.19395708600000003*TPR_Sol[4][16]*(1 - FLAG_Sol[4][16]) + 8.994956275)*FLAG_Sol[4][16] + (3.45 - (3.45*(1 - TPR_Sol[4][17])*(1 - FLAG_Sol[4][17]) + 3.45*EDLP_Sol[4][17]*FLAG_Sol[4][17]))*exp(0.19395708600000003*TPR_Sol[4][17]*(1 - FLAG_Sol[4][17]) + 8.994939622)*FLAG_Sol[4][17] + (3.45 - (3.45*(1 - TPR_Sol[4][18])*(1 - FLAG_Sol[4][18]) + 3.45*EDLP_Sol[4][18]*FLAG_Sol[4][18]))*exp(0.19395708600000003*TPR_Sol[4][18]*(1 - FLAG_Sol[4][18]) + 8.991774012)*FLAG_Sol[4][18] + (3.45 - (3.45*(1 - TPR_Sol[4][19])*(1 - FLAG_Sol[4][19]) + 3.45*EDLP_Sol[4][19]*FLAG_Sol[4][19]))*exp(0.19395708600000003*TPR_Sol[4][19]*(1 - FLAG_Sol[4][19]) + 8.986545419)*FLAG_Sol[4][19] + (3.45 - (3.45*(1 - TPR_Sol[4][20])*(1 - FLAG_Sol[4][20]) + 3.45*EDLP_Sol[4][20]*FLAG_Sol[4][20]))*exp(0.19395708600000003*TPR_Sol[4][20]*(1 - FLAG_Sol[4][20]) + 8.98313209)*FLAG_Sol[4][20] + (3.45 - (3.45*(1 - TPR_Sol[4][21])*(1 - FLAG_Sol[4][21]) + 3.45*EDLP_Sol[4][21]*FLAG_Sol[4][21]))*exp(0.19395708600000003*TPR_Sol[4][21]*(1 - FLAG_Sol[4][21]) + 8.980915319)*FLAG_Sol[4][21] + (3.45 - (3.45*(1 - TPR_Sol[4][22])*(1 - FLAG_Sol[4][22]) + 3.45*EDLP_Sol[4][22]*FLAG_Sol[4][22]))*exp(0.19395708600000003*TPR_Sol[4][22]*(1 - FLAG_Sol[4][22]) + 8.980430768)*FLAG_Sol[4][22] + (3.45 - (3.45*(1 - TPR_Sol[4][23])*(1 - FLAG_Sol[4][23]) + 3.45*EDLP_Sol[4][23]*FLAG_Sol[4][23]))*exp(0.19395708600000003*TPR_Sol[4][23]*(1 - FLAG_Sol[4][23]) + 8.978485573)*FLAG_Sol[4][23] + (3.45 - (3.45*(1 - TPR_Sol[4][24])*(1 - FLAG_Sol[4][24]) + 3.45*EDLP_Sol[4][24]*FLAG_Sol[4][24]))*exp(0.19395708600000003*TPR_Sol[4][24]*(1 - FLAG_Sol[4][24]) + 8.978042012000001)*FLAG_Sol[4][24] + (3.45 - (3.45*(1 - TPR_Sol[4][25])*(1 - FLAG_Sol[4][25]) + 3.45*EDLP_Sol[4][25]*FLAG_Sol[4][25]))*exp(0.19395708600000003*TPR_Sol[4][25]*(1 - FLAG_Sol[4][25]) + 8.978042012000001)*FLAG_Sol[4][25] + (3.45 - (3.45*(1 - TPR_Sol[4][26])*(1 - FLAG_Sol[4][26]) + 3.45*EDLP_Sol[4][26]*FLAG_Sol[4][26]))*exp(0.19395708600000003*TPR_Sol[4][26]*(1 - FLAG_Sol[4][26]) + 8.977696588999999)*FLAG_Sol[4][26] + (3.45 - (3.45*(1 - TPR_Sol[4][27])*(1 - FLAG_Sol[4][27]) + 3.45*EDLP_Sol[4][27]*FLAG_Sol[4][27]))*exp(0.19395708600000003*TPR_Sol[4][27]*(1 - FLAG_Sol[4][27]) + 8.977461638)*FLAG_Sol[4][27] + (3.45 - (3.45*(1 - TPR_Sol[4][28])*(1 - FLAG_Sol[4][28]) + 3.45*EDLP_Sol[4][28]*FLAG_Sol[4][28]))*exp(0.19395708600000003*TPR_Sol[4][28]*(1 - FLAG_Sol[4][28]) + 8.976979471)*FLAG_Sol[4][28] + (3.45 - (3.45*(1 - TPR_Sol[4][29])*(1 - FLAG_Sol[4][29]) + 3.45*EDLP_Sol[4][29]*FLAG_Sol[4][29]))*exp(0.19395708600000003*TPR_Sol[4][29]*(1 - FLAG_Sol[4][29]) + 8.976979471)*FLAG_Sol[4][29] + (3.45 - (3.45*(1 - TPR_Sol[4][30])*(1 - FLAG_Sol[4][30]) + 3.45*EDLP_Sol[4][30]*FLAG_Sol[4][30]))*exp(0.19395708600000003*TPR_Sol[4][30]*(1 - FLAG_Sol[4][30]) + 8.974200212000001)*FLAG_Sol[4][30] + (3.45 - (3.45*(1 - TPR_Sol[4][31])*(1 - FLAG_Sol[4][31]) + 3.45*EDLP_Sol[4][31]*FLAG_Sol[4][31]))*exp(0.19395708600000003*TPR_Sol[4][31]*(1 - FLAG_Sol[4][31]) + 8.973851118999999)*FLAG_Sol[4][31] + (3.45 - (3.45*(1 - TPR_Sol[4][32])*(1 - FLAG_Sol[4][32]) + 3.45*EDLP_Sol[4][32]*FLAG_Sol[4][32]))*exp(0.19395708600000003*TPR_Sol[4][32]*(1 - FLAG_Sol[4][32]) + 8.973113708)*FLAG_Sol[4][32] + (3.45 - (3.45*(1 - TPR_Sol[4][33])*(1 - FLAG_Sol[4][33]) + 3.45*EDLP_Sol[4][33]*FLAG_Sol[4][33]))*exp(0.19395708600000003*TPR_Sol[4][33]*(1 - FLAG_Sol[4][33]) + 8.971054192999999)*FLAG_Sol[4][33] + (3.45 - (3.45*(1 - TPR_Sol[4][34])*(1 - FLAG_Sol[4][34]) + 3.45*EDLP_Sol[4][34]*FLAG_Sol[4][34]))*exp(0.19395708600000003*TPR_Sol[4][34]*(1 - FLAG_Sol[4][34]) + 8.969676369)*FLAG_Sol[4][34] + (3.45 - (3.45*(1 - TPR_Sol[4][35])*(1 - FLAG_Sol[4][35]) + 3.45*EDLP_Sol[4][35]*FLAG_Sol[4][35]))*exp(0.19395708600000003*TPR_Sol[4][35]*(1 - FLAG_Sol[4][35]) + 8.969267737000001)*FLAG_Sol[4][35] + (3.45 - (3.45*(1 - TPR_Sol[4][36])*(1 - FLAG_Sol[4][36]) + 3.45*EDLP_Sol[4][36]*FLAG_Sol[4][36]))*exp(0.19395708600000003*TPR_Sol[4][36]*(1 - FLAG_Sol[4][36]) + 8.967461344)*FLAG_Sol[4][36] + (3.45 - (3.45*(1 - TPR_Sol[4][37])*(1 - FLAG_Sol[4][37]) + 3.45*EDLP_Sol[4][37]*FLAG_Sol[4][37]))*exp(0.19395708600000003*TPR_Sol[4][37]*(1 - FLAG_Sol[4][37]) + 8.967420946)*FLAG_Sol[4][37] + (3.45 - (3.45*(1 - TPR_Sol[4][38])*(1 - FLAG_Sol[4][38]) + 3.45*EDLP_Sol[4][38]*FLAG_Sol[4][38]))*exp(0.19395708600000003*TPR_Sol[4][38]*(1 - FLAG_Sol[4][38]) + 8.965657287)*FLAG_Sol[4][38] + (3.45 - (3.45*(1 - TPR_Sol[4][39])*(1 - FLAG_Sol[4][39]) + 3.45*EDLP_Sol[4][39]*FLAG_Sol[4][39]))*exp(0.19395708600000003*TPR_Sol[4][39]*(1 - FLAG_Sol[4][39]) + 8.963872594)*FLAG_Sol[4][39] + (3.45 - (3.45*(1 - TPR_Sol[4][40])*(1 - FLAG_Sol[4][40]) + 3.45*EDLP_Sol[4][40]*FLAG_Sol[4][40]))*exp(0.19395708600000003*TPR_Sol[4][40]*(1 - FLAG_Sol[4][40]) + 8.959356174)*FLAG_Sol[4][40] + (3.45 - (3.45*(1 - TPR_Sol[4][41])*(1 - FLAG_Sol[4][41]) + 3.45*EDLP_Sol[4][41]*FLAG_Sol[4][41]))*exp(0.19395708600000003*TPR_Sol[4][41]*(1 - FLAG_Sol[4][41]) + 8.959204106)*FLAG_Sol[4][41] + (3.45 - (3.45*(1 - TPR_Sol[4][42])*(1 - FLAG_Sol[4][42]) + 3.45*EDLP_Sol[4][42]*FLAG_Sol[4][42]))*exp(0.19395708600000003*TPR_Sol[4][42]*(1 - FLAG_Sol[4][42]) + 8.95579277)*FLAG_Sol[4][42] + (3.45 - (3.45*(1 - TPR_Sol[4][43])*(1 - FLAG_Sol[4][43]) + 3.45*EDLP_Sol[4][43]*FLAG_Sol[4][43]))*exp(0.19395708600000003*TPR_Sol[4][43]*(1 - FLAG_Sol[4][43]) + 8.955629276)*FLAG_Sol[4][43] + (3.45 - (3.45*(1 - TPR_Sol[4][44])*(1 - FLAG_Sol[4][44]) + 3.45*EDLP_Sol[4][44]*FLAG_Sol[4][44]))*exp(0.19395708600000003*TPR_Sol[4][44]*(1 - FLAG_Sol[4][44]) + 8.955489192)*FLAG_Sol[4][44] + (3.45 - (3.45*(1 - TPR_Sol[4][45])*(1 - FLAG_Sol[4][45]) + 3.45*EDLP_Sol[4][45]*FLAG_Sol[4][45]))*exp(0.19395708600000003*TPR_Sol[4][45]*(1 - FLAG_Sol[4][45]) + 8.953616143)*FLAG_Sol[4][45] + (3.45 - (3.45*(1 - TPR_Sol[4][46])*(1 - FLAG_Sol[4][46]) + 3.45*EDLP_Sol[4][46]*FLAG_Sol[4][46]))*exp(0.19395708600000003*TPR_Sol[4][46]*(1 - FLAG_Sol[4][46]) + 8.952622198)*FLAG_Sol[4][46] + (3.45 - (3.45*(1 - TPR_Sol[4][47])*(1 - FLAG_Sol[4][47]) + 3.45*EDLP_Sol[4][47]*FLAG_Sol[4][47]))*exp(0.19395708600000003*TPR_Sol[4][47]*(1 - FLAG_Sol[4][47]) + 8.954445913999999)*FLAG_Sol[4][47] + (3.45 - (3.45*(1 - TPR_Sol[4][48])*(1 - FLAG_Sol[4][48]) + 3.45*EDLP_Sol[4][48]*FLAG_Sol[4][48]))*exp(0.19395708600000003*TPR_Sol[4][48]*(1 - FLAG_Sol[4][48]) + 8.952622198)*FLAG_Sol[4][48] + (3.45 - (3.45*(1 - TPR_Sol[4][49])*(1 - FLAG_Sol[4][49]) + 3.45*EDLP_Sol[4][49]*FLAG_Sol[4][49]))*exp(0.19395708600000003*TPR_Sol[4][49]*(1 - FLAG_Sol[4][49]) + 8.952049938)*FLAG_Sol[4][49] + (3.45 - (3.45*(1 - TPR_Sol[4][50])*(1 - FLAG_Sol[4][50]) + 3.45*EDLP_Sol[4][50]*FLAG_Sol[4][50]))*exp(0.19395708600000003*TPR_Sol[4][50]*(1 - FLAG_Sol[4][50]) + 8.951489293)*FLAG_Sol[4][50] + (3.45 - (3.45*(1 - TPR_Sol[4][51])*(1 - FLAG_Sol[4][51]) + 3.45*EDLP_Sol[4][51]*FLAG_Sol[4][51]))*exp(0.19395708600000003*TPR_Sol[4][51]*(1 - FLAG_Sol[4][51]) + 8.950649582999999)*FLAG_Sol[4][51]


19798.300500139387

In [359]:
Solution_Wo_Optimizer

,Dollar_Sales,Units_sold,EDLP_Spend,TPR_Spend,Flag,EDLP_Dollar_Sales,TPR_Dollar_Sales,Competetor_Effect,EDLP_Multiplicative_Effect,TPR_Multiplicative_Effect,Self_Effect
P1_W1,11835.069373,5977.307764,59.773078,0.0,1,11835.069373,0.0,0.993236,0.998767,1,6025.447889
P2_W1,36456.747382,12071.770656,362.153120,0.0,1,36456.747382,0.0,1.000000,1.000000,1,12071.770656
P3_W1,10011.191137,2836.031484,0.000000,0.0,1,10011.191137,0.0,1.000000,1.000000,1,2836.031484
P4_W1,35718.178065,11866.504340,0.000000,0.0,1,35718.178065,0.0,1.000000,1.000000,1,11866.504340
P5_W1,27233.127272,8009.743315,400.487166,0.0,1,27233.127272,0.0,1.000000,1.000000,1,8009.743315
...,...,...,...,...,...,...,...,...,...,...,...
P2_W52,37778.646088,12509.485459,375.284564,0.0,1,37778.646088,0.0,1.000000,1.000000,1,12509.485459
P3_W52,10185.440422,2885.393887,0.000000,0.0,1,10185.440422,0.0,1.000000,1.000000,1,2885.393887
P4_W52,35905.578331,11928.763565,0.000000,0.0,1,35905.578331,0.0,1.000000,1.000000,1,11928.763565
P5_W52,26223.861371,7712.900403,385.645020,0.0,1,26223.861371,0.0,1.000000,1.000000,1,7712.900403


In [340]:
a=Solution_Wo_Optimizer.sort_index()

In [365]:
a["Tot_spend"] = a["EDLP_Spend"] + a["TPR_Spend"]
b=[list(a["Tot_spend"][j:j+52]) for j in range(0,len(a["Tot_spend"]),52)]
tot=[sum(i) for i in b]
tot

[3137.7379961240426,
 18848.590569294192,
 5811.024021916505,
 9630.341506670338,
 23102.593212024633,
 0.0]

In [367]:
a["Tot_spend"] = a["EDLP_Spend"]
b=[list(a["Tot_spend"][j:j+52]) for j in range(0,len(a["Tot_spend"]),52)]
edlp=[sum(i) for i in b]
edlp

[3137.7379961240426,
 18848.590569294192,
 5119.820027651015,
 8578.147936284427,
 19798.300500139387,
 0.0]

In [370]:
a["Tot_spend"] = a["TPR_Spend"]
b=[list(a["Tot_spend"][j:j+52]) for j in range(0,len(a["Tot_spend"]),52)]
tpr=[sum(i) for i in b]
tpr

[0.0, 0.0, 691.2039942654898, 1052.1935703859115, 3304.292711885247, 0.0]